# LAM

이 노트북은 DocLayout-YOLO를 활용하여 학습지 이미지의 레이아웃을 분석하는 시스템입니다.
모델에 내장된 클래스 레이블을 사용하며, 레이아웃 분석에 집중합니다.

## 주요 기능

1. 학습지 이미지의 레이아웃 분석 (DocLayout-YOLO)
2. 분석 결과 시각화
3. TSPM 연동을 위한 데이터 준비

## 실행 환경
- Google Colab
- Python 3.7+

## 1. 환경 감지 및 패키지 설치

In [ ]:
# ============================================
# 📦 환경 감지 및 패키지 설치
# ============================================

def detect_environment():
    """현재 실행 환경을 감지합니다."""
    try:
        # Colab 환경인지 확인
        import google.colab
        return 'colab'
    except ImportError:
        # 로컬 환경
        return 'local'

def install_packages():
    """환경별 패키지 설치"""
    env = detect_environment()
    print(f"🔧 환경 설정 중... (감지된 환경: {env})")
    
    if env == 'colab':
        # Colab 환경용 설치
        print("🚀 Colab 환경에서 패키지 설치 중...")
        
        # 기본 라이브러리 설치
        !pip install -q numpy opencv-python matplotlib pillow tqdm
        
        # PDF 처리를 위한 PyMuPDF 설치
        !pip install -q PyMuPDF psutil
        
        # Tesseract OCR 설치
        !apt-get update -qq
        !apt-get install -qq -y tesseract-ocr tesseract-ocr-kor
        !pip install -q pytesseract
        
        # DocLayout-YOLO 설치
        if not os.path.exists('DocLayout-YOLO'):
            !git clone https://github.com/opendatalab/DocLayout-YOLO.git
        %cd DocLayout-YOLO
        !pip install -q -e .
        
        # API 연동 및 로깅 라이브러리
        !pip install -q requests loguru rich huggingface_hub openai
        
        # 한글 폰트 설치 (TSPM용)
        !apt-get install -y fonts-nanum
        
    else:
        # 로컬 환경용 설치 (pip만 사용)
        print("🏠 로컬 환경에서 패키지 설치 중...")
        
        import subprocess
        import sys
        
        packages = [
            'numpy', 'opencv-python', 'matplotlib', 'pillow', 'tqdm',
            'PyMuPDF', 'psutil', 'pytesseract', 'requests', 'loguru', 'rich',
            'huggingface_hub', 'torch', 'ultralytics', 'openai'
        ]
        
        for package in packages:
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
                print(f"✅ {package} 설치 완료")
            except subprocess.CalledProcessError:
                print(f"⚠️ {package} 설치 실패 - 수동 설치 필요할 수 있음")
        
        # DocLayout-YOLO 설치 (로컬 환경)
        if not os.path.exists('DocLayout-YOLO'):
            try:
                subprocess.check_call(['git', 'clone', 'https://github.com/opendatalab/DocLayout-YOLO.git'])
                print("✅ DocLayout-YOLO 클론 완료")
            except subprocess.CalledProcessError:
                print("⚠️ DocLayout-YOLO 클론 실패 - 수동 클론 필요")
        
        # DocLayout-YOLO 설치
        if os.path.exists('DocLayout-YOLO'):
            original_dir = os.getcwd()
            try:
                os.chdir('DocLayout-YOLO')
                subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-e", "."])
                print("✅ DocLayout-YOLO 설치 완료")
            except subprocess.CalledProcessError:
                print("⚠️ DocLayout-YOLO 설치 실패")
            finally:
                os.chdir(original_dir)

# 환경 감지 및 설치 실행
import os
ENV = detect_environment()  # 전역 ENV 변수 설정
install_packages()

print("✅ 모든 패키지 설치 완료!")


## 2. 필요한 라이브러리 임포트

In [ ]:
# ============================================
# 📦 라이브러리 Import 및 공통 함수 정의 (환경 독립적)
# ============================================

# 기본 라이브러리 import
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
import torch
import time
import warnings
import io  # 추가된 import - PDF 처리용
import gc  # 메모리 관리용
import atexit  # 자동 정리용
import tempfile  # 임시 파일 관리용
import psutil  # 메모리 모니터링용
import threading  # 배치 처리용
from concurrent.futures import ThreadPoolExecutor, as_completed
from PIL import Image

# 경고 메시지 필터링
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# 환경 감지 (전역 변수가 이미 설정되어 있는지 확인)
if 'ENV' not in globals():
    def detect_environment():
        """현재 실행 환경을 감지합니다."""
        try:
            import google.colab
            return 'colab'
        except ImportError:
            return 'local'
    
    ENV = detect_environment()

# 환경별 tqdm import
if ENV == 'colab':
    from tqdm.notebook import tqdm
    # Colab 전용 파일 업로드
    try:
        from google.colab import files
        print("✅ Colab 환경 감지 - files 모듈 로드됨")
    except ImportError:
        print("⚠️ Colab files 모듈 로드 실패")
        files = None
else:
    from tqdm import tqdm
    print("✅ 로컬 환경 감지 - 표준 tqdm 사용")
    files = None

# PDF 처리를 위한 PyMuPDF
try:
    import fitz  # PyMuPDF
    print("✅ PyMuPDF 사용 가능")
except ImportError:
    print("PyMuPDF가 설치되지 않았습니다. 설치 중...")
    if ENV == 'colab':
        !pip install -q PyMuPDF psutil
    else:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "PyMuPDF", "psutil"])
    import fitz
    import psutil

# DocLayout-YOLO import
try:
    from doclayout_yolo import YOLOv10
    print("✅ DocLayout-YOLO 사용 가능")
except ImportError:
    print("❌ DocLayout-YOLO 설치가 필요합니다. 이전 셀을 먼저 실행해주세요.")
    YOLOv10 = None

# 로깅 설정
from loguru import logger

# 로그 레벨 설정
logger.remove()
logger.add(sys.stderr, level="INFO")
logger.add("smarteye_analysis.log", rotation="10 MB", level="DEBUG")

# ============================================
# 📝 설정 및 상수 정의 (개선됨)
# ============================================

class Config:
    """SmartEye 시스템 설정 (개선된 버전)"""
    
    # 지원되는 파일 형식
    SUPPORTED_IMAGE_FORMATS = {'.jpg', '.jpeg', '.png', '.pdf'}
    
    # OCR 설정(수정 필요)
    OCR_TARGET_CLASSES = [
        'title', 'plain text', 'abandon text',
        'table caption', 'table footnote',
        'isolated formula', 'formula caption'
    ]
    
    OCR_CONFIG = r'--oem 3 --psm 6'
    
    # API 설정
    API_TARGET_CLASSES = ['figure', 'table']
    
    # 이미지 처리 설정
    MIN_AREA_THRESHOLD = 100  # 최소 픽셀 수
    MAX_DISPLAY_SIZE_COLAB = 800
    MAX_DISPLAY_SIZE_LOCAL = 1000
    
    # 메모리 관리 설정 (동적 조정)
    COLAB_MEMORY_LIMIT_MB = 100
    LOCAL_MEMORY_LIMIT_MB = 500
    MEMORY_WARNING_THRESHOLD = 0.8  # 80% 사용 시 경고
    MEMORY_CRITICAL_THRESHOLD = 0.9  # 90% 사용 시 중단
    
    # 배치 처리 설정
    BATCH_SIZE_COLAB = 2  # Colab용 배치 크기
    BATCH_SIZE_LOCAL = 4  # 로컬용 배치 크기
    MAX_WORKERS = 2  # 병렬 처리 워커 수
    
    # 임시 파일 설정
    TEMP_FILE_PREFIX = "smarteye_temp_"
    AUTO_CLEANUP_TEMP_FILES = True
    
    # 진행률 저장 설정
    CHECKPOINT_INTERVAL = 3  # 3개 이미지마다 체크포인트 저장
    CHECKPOINT_FILE = "smarteye_checkpoint.json"
    
    # PDF 처리 최적화
    PDF_ZOOM_FACTOR = 1.5  # 2.0에서 1.5로 줄임 (메모리 절약)
    PDF_MAX_PAGES_PER_BATCH = 3  # 한 번에 처리할 PDF 페이지 수
    
    # 모델 설정
    MODEL_CONFIGS = {
        "doclaynet_docsynth": {
            "repo_id": "juliozhao/DocLayout-YOLO-DocLayNet-Docsynth300K_pretrained",
            "filename": "doclayout_yolo_doclaynet_imgsz1120_docsynth_pretrain.pt",
            "imgsz": 1536,
            "conf": 0.20
        },
        "docstructbench": {
            "repo_id": "juliozhao/DocLayout-YOLO-DocStructBench",
            "filename": "doclayout_yolo_docstructbench_imgsz1024.pt",
            "imgsz": 1024,
            "conf": 0.25
        },
        "docsynth300k": {
            "repo_id": "juliozhao/DocLayout-YOLO-DocSynth300K-pretrain",
            "filename": "doclayout_yolo_docsynth300k_imgsz1600.pt",
            "imgsz": 1600,
            "conf": 0.15
        }
    }

# ============================================
# 🛠️ 메모리 관리 및 모니터링 시스템
# ============================================

class MemoryManager:
    """동적 메모리 관리 및 모니터링 클래스"""
    
    def __init__(self):
        self.process = psutil.Process()
        self.initial_memory = self.get_memory_usage_mb()
        
    def get_memory_usage_mb(self):
        """현재 메모리 사용량 (MB) 반환"""
        return self.process.memory_info().rss / 1024 / 1024
    
    def get_memory_percent(self):
        """시스템 메모리 사용률 (%) 반환"""
        return psutil.virtual_memory().percent / 100
    
    def check_memory_status(self):
        """메모리 상태 체크 및 경고"""
        memory_percent = self.get_memory_percent()
        current_mb = self.get_memory_usage_mb()
        
        if memory_percent > Config.MEMORY_CRITICAL_THRESHOLD:
            logger.critical(f"🔴 메모리 위험 수준: {memory_percent:.1%} (현재: {current_mb:.1f}MB)")
            return "critical"
        elif memory_percent > Config.MEMORY_WARNING_THRESHOLD:
            logger.warning(f"🟡 메모리 경고 수준: {memory_percent:.1%} (현재: {current_mb:.1f}MB)")
            return "warning"
        else:
            logger.debug(f"🟢 메모리 정상: {memory_percent:.1%} (현재: {current_mb:.1f}MB)")
            return "normal"
    
    def force_cleanup(self):
        """강제 메모리 정리"""
        logger.info("🧹 메모리 강제 정리 실행...")
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        
        after_cleanup = self.get_memory_usage_mb()
        logger.info(f"✅ 메모리 정리 완료: {after_cleanup:.1f}MB")

# 전역 메모리 매니저 인스턴스
memory_manager = MemoryManager()

# ============================================
# 🗂️ 임시 파일 관리 시스템
# ============================================

class TempFileManager:
    """임시 파일 자동 관리 클래스"""
    
    def __init__(self):
        self.temp_files = []
        self.temp_dirs = []
        self.lock = threading.Lock()
        
        # 프로그램 종료 시 자동 정리 등록
        if Config.AUTO_CLEANUP_TEMP_FILES:
            atexit.register(self.cleanup_all)
    
    def create_temp_file(self, suffix=".png", prefix=None):
        """임시 파일 생성 및 추적"""
        if prefix is None:
            prefix = Config.TEMP_FILE_PREFIX
            
        temp_file = tempfile.NamedTemporaryFile(
            delete=False, suffix=suffix, prefix=prefix
        )
        temp_file_path = temp_file.name
        temp_file.close()
        
        with self.lock:
            self.temp_files.append(temp_file_path)
        
        logger.debug(f"임시 파일 생성: {temp_file_path}")
        return temp_file_path
    
    def create_temp_dir(self, prefix=None):
        """임시 디렉토리 생성 및 추적"""
        if prefix is None:
            prefix = Config.TEMP_FILE_PREFIX
            
        temp_dir = tempfile.mkdtemp(prefix=prefix)
        
        with self.lock:
            self.temp_dirs.append(temp_dir)
        
        logger.debug(f"임시 디렉토리 생성: {temp_dir}")
        return temp_dir
    
    def remove_temp_file(self, file_path):
        """특정 임시 파일 제거"""
        try:
            if os.path.exists(file_path):
                os.remove(file_path)
                logger.debug(f"임시 파일 제거: {file_path}")
            
            with self.lock:
                if file_path in self.temp_files:
                    self.temp_files.remove(file_path)
        except Exception as e:
            logger.warning(f"임시 파일 제거 실패: {file_path} - {e}")
    
    def cleanup_all(self):
        """모든 임시 파일 및 디렉토리 정리"""
        logger.info("🧹 임시 파일 전체 정리 시작...")
        
        # 임시 파일 정리
        cleaned_files = 0
        for temp_file in self.temp_files[:]:
            try:
                if os.path.exists(temp_file):
                    os.remove(temp_file)
                    cleaned_files += 1
            except Exception as e:
                logger.warning(f"임시 파일 정리 실패: {temp_file} - {e}")
        
        # 임시 디렉토리 정리
        cleaned_dirs = 0
        for temp_dir in self.temp_dirs[:]:
            try:
                if os.path.exists(temp_dir):
                    import shutil
                    shutil.rmtree(temp_dir)
                    cleaned_dirs += 1
            except Exception as e:
                logger.warning(f"임시 디렉토리 정리 실패: {temp_dir} - {e}")
        
        self.temp_files.clear()
        self.temp_dirs.clear()
        
        logger.info(f"✅ 임시 파일 정리 완료: 파일 {cleaned_files}개, 디렉토리 {cleaned_dirs}개")

# 전역 임시 파일 매니저 인스턴스
temp_file_manager = TempFileManager()

# ============================================
# 💾 체크포인트 시스템
# ============================================

class CheckpointManager:
    """진행률 저장 및 복구 시스템"""
    
    def __init__(self, checkpoint_file=None):
        self.checkpoint_file = checkpoint_file or Config.CHECKPOINT_FILE
        self.checkpoint_data = {}
    
    def save_checkpoint(self, progress_data):
        """진행률 저장"""
        try:
            checkpoint = {
                'timestamp': time.time(),
                'progress': progress_data,
                'memory_usage_mb': memory_manager.get_memory_usage_mb()
            }
            
            with open(self.checkpoint_file, 'w', encoding='utf-8') as f:
                json.dump(checkpoint, f, ensure_ascii=False, indent=2)
            
            logger.debug(f"체크포인트 저장: {len(progress_data.get('completed', []))}개 완료")
            
        except Exception as e:
            logger.warning(f"체크포인트 저장 실패: {e}")
    
    def load_checkpoint(self):
        """저장된 진행률 로드"""
        try:
            if os.path.exists(self.checkpoint_file):
                with open(self.checkpoint_file, 'r', encoding='utf-8') as f:
                    checkpoint = json.load(f)
                
                logger.info(f"체크포인트 로드: {len(checkpoint.get('progress', {}).get('completed', []))}개 완료됨")
                return checkpoint.get('progress', {})
            
        except Exception as e:
            logger.warning(f"체크포인트 로드 실패: {e}")
        
        return {}
    
    def clear_checkpoint(self):
        """체크포인트 파일 제거"""
        try:
            if os.path.exists(self.checkpoint_file):
                os.remove(self.checkpoint_file)
                logger.info("체크포인트 파일 정리 완료")
        except Exception as e:
            logger.warning(f"체크포인트 정리 실패: {e}")

# 전역 체크포인트 매니저 인스턴스
checkpoint_manager = CheckpointManager()

# ============================================
# 🛠️ 공통 유틸리티 함수 (개선됨)
# ============================================

def safe_filename(filename, max_length=50):
    """파일명을 안전하게 처리"""
    return "".join(c for c in filename if c.isalnum() or c in '._-')[:max_length]

def ensure_directory(directory):
    """디렉토리가 존재하지 않으면 생성"""
    try:
        os.makedirs(directory, exist_ok=True)
        return True
    except Exception as e:
        logger.error(f"디렉토리 생성 실패: {directory} - {e}")
        return False

def calculate_image_stats(processed_images):
    """이미지 목록의 통계 계산"""
    if not processed_images:
        return {"count": 0, "total_size_mb": 0}
    
    total_size = sum(img.get('file_size_mb', 0) for img in processed_images)
    return {
        "count": len(processed_images),
        "total_size_mb": total_size
    }

def check_memory_limits(total_size_mb):
    """메모리 제한 체크 및 경고 (개선됨)"""
    # 동적 메모리 상태 체크
    memory_status = memory_manager.check_memory_status()
    
    if memory_status == "critical":
        logger.error("🔴 메모리 부족으로 처리를 중단합니다.")
        return False
    
    # 기존 정적 체크도 유지
    if ENV == 'colab' and total_size_mb > Config.COLAB_MEMORY_LIMIT_MB:
        logger.warning(f"⚠️ 총 이미지 크기가 {total_size_mb:.1f}MB입니다. Colab 메모리 부족이 발생할 수 있습니다.")
        return False
    elif ENV == 'local' and total_size_mb > Config.LOCAL_MEMORY_LIMIT_MB:
        logger.warning(f"⚠️ 총 이미지 크기가 {total_size_mb:.1f}MB입니다. 처리 시간이 오래 걸릴 수 있습니다.")
        return False
    
    return True

def adaptive_batch_size():
    """메모리 상태에 따른 동적 배치 크기 조정"""
    memory_percent = memory_manager.get_memory_percent()
    
    if ENV == 'colab':
        base_batch_size = Config.BATCH_SIZE_COLAB
    else:
        base_batch_size = Config.BATCH_SIZE_LOCAL
    
    if memory_percent > 0.8:
        return max(1, base_batch_size // 2)  # 메모리 부족 시 배치 크기 절반
    elif memory_percent < 0.5:
        return base_batch_size * 2  # 메모리 여유 시 배치 크기 증가
    else:
        return base_batch_size

# 환경 정보 출력
print(f"📍 현재 실행 환경: {ENV.upper()}")
print(f"💾 초기 메모리 사용량: {memory_manager.get_memory_usage_mb():.1f}MB")
print(f"🖥️ 시스템 메모리 사용률: {memory_manager.get_memory_percent():.1%}")
print("🚀 개선된 라이브러리 로딩 및 설정 완료!")

## 3. 사전 훈련된 모델 다운로드 함수 정의

In [ ]:
def download_model(model_choice="docstructbench"):
    """사전 훈련된 DocLayout-YOLO 모델 다운로드

    Args:
        model_choice (str): 사용할 모델 유형 (Config.MODEL_CONFIGS의 키)

    Returns:
        str: 다운로드된 모델 파일 경로
        
    Raises:
        ValueError: 지원하지 않는 모델 선택 시
        Exception: 다운로드 실패 시
    """
    try:
        from huggingface_hub import hf_hub_download
    except ImportError:
        logger.info("huggingface_hub 설치 중...")
        if ENV == 'colab':
            !pip install -q huggingface_hub
        else:
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "huggingface_hub"])
        from huggingface_hub import hf_hub_download

    # 모델 설정 가져오기
    if model_choice not in Config.MODEL_CONFIGS:
        available_models = ", ".join(Config.MODEL_CONFIGS.keys())
        raise ValueError(f"지원하지 않는 모델: {model_choice}. 사용 가능한 모델: {available_models}")
    
    selected_model = Config.MODEL_CONFIGS[model_choice]

    try:
        logger.info(f"다운로드 중: {selected_model['repo_id']} - {selected_model['filename']}")

        filepath = hf_hub_download(
            repo_id=selected_model["repo_id"],
            filename=selected_model["filename"]
        )

        logger.success(f"모델이 다운로드 되었습니다: {filepath}")
        return filepath
        
    except Exception as e:
        logger.error(f"모델 다운로드 중 오류 발생: {e}")
        raise RuntimeError(f"모델 다운로드 실패: {model_choice}") from e

## 4. 이미지 업로드 및 전처리 컴포넌트

In [ ]:
# ============================================
# 📁 이미지 업로드 및 전처리 컴포넌트 (최적화됨)
# ============================================

def upload_and_prepare_images():
    """환경에 따라 적절한 이미지 업로드 방법을 사용합니다."""
    logger.info(f"감지된 환경: {ENV}")
    
    if ENV == 'colab':
        return upload_and_prepare_images_colab()
    else:
        return upload_and_prepare_images_local()

def upload_and_prepare_images_colab():
    """Colab 환경용 다중 이미지 업로드 및 다양한 포맷 지원"""
    logger.info("다중 이미지 업로드 대기 중...")
    
    try:
        if files is None:
            logger.error("Colab files 모듈을 사용할 수 없습니다.")
            return None
        
        # 사용자 안내
        print("📁 지원 파일 형식:", ", ".join(Config.SUPPORTED_IMAGE_FORMATS))
        print("💡 Ctrl+클릭으로 여러 파일을 한 번에 선택할 수 있습니다.")
        print("📄 PDF의 경우 모든 페이지가 자동으로 개별 이미지로 변환됩니다.")
        
        uploaded = files.upload()
        
        if not uploaded:
            logger.error("이미지가 업로드되지 않았습니다.")
            return None
        
        return process_uploaded_files(uploaded)
        
    except Exception as e:
        logger.error(f"이미지 업로드 중 오류 발생: {e}")
        return None

def upload_and_prepare_images_local():
    """로컬 환경용 파일 선택 및 이미지 업로드"""
    logger.info("로컬 환경에서 이미지 업로드 대기 중...")
    
    try:
        import tkinter as tk
        from tkinter import filedialog
        
        # tkinter 루트 윈도우 생성 (숨김)
        root = tk.Tk()
        root.withdraw()
        
        # 파일 형식 문자열 생성
        format_string = " ".join(f"*{fmt}" for fmt in Config.SUPPORTED_IMAGE_FORMATS)
        
        # 파일 선택 대화상자
        file_paths = filedialog.askopenfilenames(
            title="학습지 이미지 선택 (여러 파일 선택 가능)",
            filetypes=[
                ("이미지 파일", format_string),
                ("JPG 파일", "*.jpg *.jpeg"),
                ("PNG 파일", "*.png"),
                ("PDF 파일", "*.pdf"),
                ("모든 파일", "*.*")
            ]
        )
        
        if not file_paths:
            logger.error("파일이 선택되지 않았습니다.")
            return None
        
        # 파일 경로를 딕셔너리로 변환 (Colab 형식과 일치)
        uploaded = {os.path.basename(path): None for path in file_paths}
        
        return process_uploaded_files(uploaded, file_paths)
        
    except Exception as e:
        logger.error(f"로컬 환경에서 이미지 업로드 중 오류 발생: {e}")
        return None

def process_uploaded_files(uploaded, file_paths=None):
    """업로드된 파일들을 처리 (개선된 메모리 관리)"""
    processed_images = []
    
    logger.info(f"업로드된 파일 수: {len(uploaded)}개")
    
    for idx, filename in enumerate(uploaded.keys()):
        logger.info(f"처리 중인 파일: {filename}")
        
        # 메모리 상태 체크
        memory_status = memory_manager.check_memory_status()
        if memory_status == "critical":
            logger.error("메모리 부족으로 파일 처리를 중단합니다.")
            break
        
        # 파일 확장자 확인
        file_ext = os.path.splitext(filename)[1].lower()
        
        if file_ext not in Config.SUPPORTED_IMAGE_FORMATS:
            logger.warning(f"지원하지 않는 파일 형식: {filename}")
            continue
        
        # 실제 파일 경로 결정
        if file_paths:
            actual_path = file_paths[idx] if idx < len(file_paths) else filename
        else:
            actual_path = filename
        
        try:
            # PDF 처리 (스트리밍 방식)
            if file_ext == '.pdf':
                pdf_images = process_pdf_file_streaming(actual_path)
                processed_images.extend(pdf_images)
            # 이미지 파일 처리
            else:
                img_info = process_image_file(actual_path)
                if img_info:
                    processed_images.append(img_info)
                    
        except Exception as e:
            logger.error(f"파일 처리 중 오류: {filename} - {e}")
            continue
        
        # 주기적 메모리 정리
        if idx % 2 == 1:  # 2개 파일마다
            memory_manager.force_cleanup()
    
    logger.info(f"총 {len(processed_images)}개의 이미지가 처리되었습니다.")
    
    # 메모리 체크
    stats = calculate_image_stats(processed_images)
    check_memory_limits(stats["total_size_mb"])
    
    # 처리된 이미지들 미리보기 (적응형)
    if ENV == 'colab':
        display_image_grid_colab(processed_images)
    else:
        display_image_grid_local(processed_images)
    
    return processed_images

def process_pdf_file_streaming(pdf_path):
    """PDF 파일을 스트리밍 방식으로 처리 (메모리 최적화)"""
    pdf_images = []
    
    try:
        # PDF 문서 열기
        pdf_document = fitz.open(pdf_path)
        total_pages = len(pdf_document)
        logger.info(f"PDF 페이지 수: {total_pages}페이지")
        
        # 배치 단위로 처리
        batch_size = Config.PDF_MAX_PAGES_PER_BATCH
        
        for batch_start in range(0, total_pages, batch_size):
            batch_end = min(batch_start + batch_size, total_pages)
            logger.info(f"PDF 배치 처리: {batch_start + 1}-{batch_end} 페이지")
            
            # 메모리 상태 체크
            memory_status = memory_manager.check_memory_status()
            if memory_status == "critical":
                logger.error("메모리 부족으로 PDF 처리를 중단합니다.")
                break
            
            batch_images = []
            
            for page_num in range(batch_start, batch_end):
                try:
                    page = pdf_document[page_num]
                    
                    # 최적화된 zoom factor 사용
                    zoom = Config.PDF_ZOOM_FACTOR
                    mat = fitz.Matrix(zoom, zoom)
                    pix = page.get_pixmap(matrix=mat)
                    
                    # PIL Image로 변환
                    img_data = pix.tobytes("png")
                    pil_img = Image.open(io.BytesIO(img_data))
                    
                    # OpenCV 형식으로 변환
                    img_array = np.array(pil_img)
                    if len(img_array.shape) == 3:
                        img = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
                    else:
                        img = cv2.cvtColor(img_array, cv2.COLOR_GRAY2BGR)
                    
                    # 임시 파일 생성 (관리되는 방식)
                    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
                    temp_filename = temp_file_manager.create_temp_file(
                        suffix=f"_page_{page_num + 1}.png",
                        prefix=f"pdf_{safe_filename(base_name)}_"
                    )
                    
                    # 임시 파일에 저장
                    success = cv2.imwrite(temp_filename, img)
                    
                    if success:
                        batch_images.append({
                            'original_filename': os.path.basename(pdf_path),
                            'processed_filename': temp_filename,
                            'page_number': page_num + 1,
                            'img_array': img,
                            'source_type': 'pdf',
                            'file_size_mb': len(img_data) / (1024 * 1024)
                        })
                        
                        logger.info(f"PDF 페이지 {page_num + 1}/{total_pages} 변환 완료")
                    else:
                        logger.error(f"PDF 페이지 {page_num + 1} 저장 실패")
                    
                    # 즉시 메모리 정리
                    pix = None
                    pil_img = None
                    img_data = None
                    del img_data, pil_img, pix
                    
                except Exception as e:
                    logger.error(f"PDF 페이지 {page_num + 1} 처리 중 오류: {e}")
                    continue
            
            # 배치 완료 후 메모리 정리
            pdf_images.extend(batch_images)
            memory_manager.force_cleanup()
            
            logger.info(f"PDF 배치 {batch_start + 1}-{batch_end} 완료 ({len(batch_images)}개 페이지)")
        
        # PDF 문서 닫기
        pdf_document.close()
        
    except Exception as e:
        logger.error(f"PDF 처리 중 오류 발생: {e}")
        if 'pdf_document' in locals():
            pdf_document.close()
    
    return pdf_images

def process_image_file(image_path):
    """단일 이미지 파일 처리"""
    try:
        if not os.path.exists(image_path):
            logger.error(f"파일이 존재하지 않습니다: {image_path}")
            return None
            
        # OpenCV로 이미지 로드
        img = cv2.imread(image_path)
        
        if img is None:
            logger.error(f"이미지를 로드할 수 없습니다: {image_path}")
            return None
        
        # 파일 크기 확인
        file_size = os.path.getsize(image_path) / (1024 * 1024)  # MB 단위
        
        return {
            'original_filename': os.path.basename(image_path),
            'processed_filename': image_path,
            'page_number': 1,
            'img_array': img,
            'source_type': 'image',
            'file_size_mb': file_size
        }
        
    except Exception as e:
        logger.error(f"이미지 처리 중 오류 발생: {e}")
        return None

def display_image_grid_colab(processed_images, max_display=6):
    """Colab 환경 최적화 이미지 격자 미리보기 (적응형)"""
    if not processed_images:
        return
    
    # 메모리 상태에 따른 표시 개수 조정
    memory_status = memory_manager.check_memory_status()
    if memory_status == "warning":
        max_display = min(max_display, 3)
    elif memory_status == "critical":
        max_display = min(max_display, 1)
    
    display_images = processed_images[:max_display]
    num_images = len(display_images)
    
    cols = min(2, num_images)
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(12, 4 * rows))
    
    if num_images == 1:
        axes = [axes]
    elif rows == 1 and cols > 1:
        axes = axes if hasattr(axes, '__len__') else [axes]
    elif rows > 1:
        axes = axes.flatten()
    
    for i, img_info in enumerate(display_images):
        img_display = img_info['img_array']
        height, width = img_display.shape[:2]
        
        # 이미지 리사이즈
        if max(height, width) > Config.MAX_DISPLAY_SIZE_COLAB:
            scale = Config.MAX_DISPLAY_SIZE_COLAB / max(height, width)
            new_width = int(width * scale)
            new_height = int(height * scale)
            img_display = cv2.resize(img_display, (new_width, new_height))
        
        img_rgb = cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB)
        
        ax = axes[i] if num_images > 1 else axes[0]
        ax.imshow(img_rgb)
        ax.axis('off')
        
        # 제목 설정
        if img_info['source_type'] == 'pdf':
            title = f"{img_info['original_filename']}\n(Page {img_info['page_number']}, {img_info['file_size_mb']:.1f}MB)"
        else:
            title = f"{img_info['original_filename']}\n({img_info['file_size_mb']:.1f}MB)"
        
        ax.set_title(title, fontsize=9)
    
    # 빈 subplot 숨기기
    if num_images > 1:
        for i in range(num_images, len(axes)):
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    if len(processed_images) > max_display:
        logger.info(f"총 {len(processed_images)}개 이미지 중 {max_display}개만 미리보기로 표시됨")

def display_image_grid_local(processed_images, max_display=6):
    """로컬 환경용 이미지 격자 미리보기 (적응형)"""
    if not processed_images:
        return
    
    # 메모리 상태에 따른 표시 개수 조정
    memory_status = memory_manager.check_memory_status()
    if memory_status == "warning":
        max_display = min(max_display, 4)
    elif memory_status == "critical":
        max_display = min(max_display, 2)
    
    display_images = processed_images[:max_display]
    num_images = len(display_images)
    
    cols = min(3, num_images)
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    
    if num_images == 1:
        axes = [axes]
    elif rows == 1 and cols > 1:
        axes = axes if hasattr(axes, '__len__') else [axes]
    elif rows > 1:
        axes = axes.flatten()
    
    for i, img_info in enumerate(display_images):
        img_display = img_info['img_array']
        height, width = img_display.shape[:2]
        
        # 이미지 리사이즈
        if max(height, width) > Config.MAX_DISPLAY_SIZE_LOCAL:
            scale = Config.MAX_DISPLAY_SIZE_LOCAL / max(height, width)
            new_width = int(width * scale)
            new_height = int(height * scale)
            img_display = cv2.resize(img_display, (new_width, new_height))
        
        img_rgb = cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB)
        
        ax = axes[i] if num_images > 1 else axes[0]
        ax.imshow(img_rgb)
        ax.axis('off')
        
        # 제목 설정
        if img_info['source_type'] == 'pdf':
            title = f"{img_info['original_filename']}\n(Page {img_info['page_number']}, {img_info['file_size_mb']:.1f}MB)"
        else:
            title = f"{img_info['original_filename']}\n({img_info['file_size_mb']:.1f}MB)"
        
        ax.set_title(title, fontsize=10)
    
    # 빈 subplot 숨기기
    if num_images > 1:
        for i in range(num_images, len(axes)):
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    if len(processed_images) > max_display:
        logger.info(f"총 {len(processed_images)}개 이미지 중 {max_display}개만 미리보기로 표시됨")

# 기존 단일 이미지 업로드 함수 (호환성 유지)
def upload_and_prepare_image():
    """이미지 업로드 및 기본 시각화 (기존 호환성용)"""
    logger.info("단일 이미지 업로드 대기 중...")
    
    if ENV == 'colab':
        try:
            if files is None:
                logger.error("Colab files 모듈을 사용할 수 없습니다.")
                return None, None
                
            uploaded = files.upload()
            
            if not uploaded:
                logger.error("이미지가 업로드되지 않았습니다.")
                return None, None
            
            image_path = list(uploaded.keys())[0]
            logger.info(f"업로드된 이미지: {image_path}")
            
            # 이미지 로드 및 시각화
            img = cv2.imread(image_path)
            
            if img is None:
                logger.error(f"이미지를 로드할 수 없습니다: {image_path}")
                return None, None
            
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            plt.figure(figsize=(12, 10), dpi=150)
            plt.imshow(img_rgb)
            plt.axis('off')
            plt.title('업로드된 학습지 이미지')
            plt.show()
            
            return image_path, img
            
        except Exception as e:
            logger.error(f"Colab 환경에서 이미지 업로드 중 오류 발생: {e}")
            return None, None
    
    else:  # 로컬 환경
        try:
            import tkinter as tk
            from tkinter import filedialog
            
            root = tk.Tk()
            root.withdraw()
            
            image_path = filedialog.askopenfilename(
                title="학습지 이미지 선택",
                filetypes=[
                    ("이미지 파일", "*.jpg *.jpeg *.png"),
                    ("JPG 파일", "*.jpg"),
                    ("PNG 파일", "*.png"),
                    ("모든 파일", "*.*")
                ]
            )
            
            if not image_path:
                logger.error("파일이 선택되지 않았습니다.")
                return None, None
            
            logger.info(f"선택된 이미지: {image_path}")
            
            # 이미지 로드 및 시각화
            img = cv2.imread(image_path)
            
            if img is None:
                logger.error(f"이미지를 로드할 수 없습니다: {image_path}")
                return None, None
            
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            plt.figure(figsize=(12, 10), dpi=150)
            plt.imshow(img_rgb)
            plt.axis('off')
            plt.title('선택된 학습지 이미지')
            plt.show()
            
            return image_path, img
            
        except Exception as e:
            logger.error(f"로컬 환경에서 이미지 업로드 중 오류 발생: {e}")
            return None, None

## 5. 모델 로드 및 설정

In [ ]:
def load_model(model_path):
    """DocLayout-YOLO 모델 로드"""
    try:
        device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        logger.info(f"사용 중인 디바이스: {device}")

        # 중요: 추론 모드로만 모델 로드 (두 번째 매개변수로 task 지정)
        model = YOLOv10(model_path, task='predict')

        # 디바이스 설정
        model.to(device)

        # 주의: model.eval() 호출하지 않음
        # 대신 필요한 경우 직접 training 속성 설정
        if hasattr(model, 'training'):
            model.training = False

        return model, device

    except Exception as e:
        logger.error(f"모델 로드 중 오류 발생: {e}")
        raise

## 6. 레이아웃 분석 컴포넌트

In [ ]:
def analyze_layout(model, image_path, img, device='cpu', model_choice="docstructbench"):
    """DocLayout-YOLO를 사용한 레이아웃 분석"""
    logger.info("레이아웃 분석 시작...")

    try:
        # 모델 설정 가져오기
        if model_choice not in Config.MODEL_CONFIGS:
            logger.warning(f"알 수 없는 모델: {model_choice}, 기본값 사용")
            model_choice = "docstructbench"
        
        model_config = Config.MODEL_CONFIGS[model_choice]
        imgsz = model_config["imgsz"]
        conf = model_config["conf"]
        
        logger.info(f"모델 설정: {model_choice} (imgsz={imgsz}, conf={conf})")

        # 분석 결과 얻기
        results = model.predict(
            image_path,
            imgsz=imgsz,
            conf=conf,
            iou=0.45,
            device=device
        )

        # 결과 검증
        if not results or len(results) == 0:
            logger.warning("모델에서 결과를 반환하지 않았습니다.")
            return []

        result = results[0]
        if not hasattr(result, 'boxes') or result.boxes is None:
            logger.warning("감지된 객체가 없습니다.")
            return []

        # 감지된 영역 정보 추출
        boxes = result.boxes.xyxy.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()
        class_names = model.names

        logger.debug(f"모델이 인식한 클래스 이름: {class_names}")
        logger.info(f"감지된 원시 객체 수: {len(boxes)}")

        # 레이아웃 정보 구성
        layout_info = []
        filtered_count = 0

        for i, (box, cls, conf) in enumerate(zip(boxes, classes, confs)):
            x1, y1, x2, y2 = map(int, box)
            cls_id = int(cls)

            try:
                cls_name = class_names[cls_id]
            except (IndexError, KeyError):
                logger.warning(f"인덱스 {cls_id}에 해당하는 클래스 이름을 찾을 수 없습니다.")
                cls_name = f"unknown_{cls_id}"

            # 면적 기반 필터링
            area = (x2 - x1) * (y2 - y1)
            if area < Config.MIN_AREA_THRESHOLD:
                logger.debug(f"작은 영역 무시: {cls_name}, 면적: {area}px")
                filtered_count += 1
                continue

            # 좌표 유효성 검사
            img_height, img_width = img.shape[:2]
            if x1 < 0 or y1 < 0 or x2 > img_width or y2 > img_height:
                logger.debug(f"범위를 벗어난 좌표 보정: {cls_name}")
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(img_width, x2)
                y2 = min(img_height, y2)

            # 이미지 크기 대비 영역 비율 계산
            width_ratio = (x2 - x1) / img_width
            height_ratio = (y2 - y1) / img_height

            layout_info.append({
                'id': i,
                'class_name': cls_name,
                'confidence': float(conf),
                'box': [int(x1), int(y1), int(x2), int(y2)],
                'coordinates': [int(x1), int(y1), int(x2), int(y2)],  # 호환성용
                'width': int(x2 - x1),
                'height': int(y2 - y1),
                'area': area,
                'width_ratio': width_ratio,
                'height_ratio': height_ratio,
                'center_x': int((x1 + x2) / 2),
                'center_y': int((y1 + y2) / 2)
            })

        # 통계 출력
        unique_classes = set(layout['class_name'] for layout in layout_info)
        logger.info(f"필터링 후 {len(layout_info)}개의 레이아웃 영역 (필터링됨: {filtered_count}개)")
        logger.info(f"감지된 고유 클래스 수: {len(unique_classes)}개")
        logger.info(f"감지된 클래스: {unique_classes}")

        return layout_info

    except Exception as e:
        logger.error(f"레이아웃 분석 중 오류 발생: {e}")
        import traceback
        logger.error(traceback.format_exc())
        raise RuntimeError(f"레이아웃 분석 실패") from e

## 7. 시각화 컴포넌트

In [ ]:
def visualize_results(img, layout_info, alpha=0.2):
    """결과 시각화 - 레이아웃 분석 결과만 표시"""
    logger.info("분석 결과 시각화 중...")

    try:
        # 원본 이미지 복사
        img_result = img.copy()
        overlay = img.copy()  # 반투명 오버레이용 이미지

        # 클래스별 색상 생성
        import random
        import colorsys
        random.seed(42)  # 일관된 색상 배정을 위한 시드 설정

        # 모든 고유 클래스 수집
        unique_classes = list(set(layout['class_name'] for layout in layout_info))

        # 클래스별 색상 생성 (HSV 색상 공간에서 균일하게 분포)
        class_colors = {}
        for i, cls_name in enumerate(unique_classes):
            h = i / max(1, len(unique_classes))  # 0~1 사이 값
            s = 0.8  # 채도
            v = 0.9  # 명도

            # HSV -> RGB -> BGR 변환
            r, g, b = colorsys.hsv_to_rgb(h, s, v)
            class_colors[cls_name] = (int(b * 255), int(g * 255), int(r * 255))

        logger.info(f"감지된 클래스: {len(unique_classes)}개 - {', '.join(unique_classes)}")

        # 클래스별 카운터 초기화
        class_counters = {cls: 0 for cls in unique_classes}

        # 개별 레이아웃 요소 시각화
        for layout in layout_info:
            x1, y1, x2, y2 = layout['box']
            cls_name = layout['class_name']

            # 클래스 카운터 증가
            class_counters[cls_name] += 1
            class_id = class_counters[cls_name]

            # 클래스별 색상 가져오기
            color = class_colors[cls_name]

            # 1. 바운딩 박스 내부를 반투명하게 색칠
            cv2.rectangle(overlay, (x1, y1), (x2, y2), color, -1)  # -1은 내부 채우기

            # 2. 바운딩 박스 테두리 그리기 (실선)
            cv2.rectangle(img_result, (x1, y1), (x2, y2), color, 2)

            # 3. 라벨 텍스트에 클래스 이름과 클래스별 ID 표시
            label = f"{cls_name} {class_id} ({layout['confidence']:.2f})"

            # 라벨 배경
            labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            y1_label = max(y1, labelSize[1] + 10)
            cv2.rectangle(
                img_result,
                (x1, y1_label - labelSize[1] - 10),
                (x1 + labelSize[0], y1_label),
                color,
                -1
            )

            # 라벨 텍스트
            cv2.putText(
                img_result,
                label,
                (x1, y1_label - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 255),
                1
            )

        # 반투명 오버레이 적용
        img_result = cv2.addWeighted(overlay, alpha, img_result, 1 - alpha, 0)

        # RGB 변환 (Matplotlib 표시용)
        img_rgb = cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB)

        # 결과 시각화
        plt.figure(figsize=(15, 12))
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title('학습지 레이아웃 분석 결과')
        plt.tight_layout()

        # 범례 생성 (검출된 클래스만)
        import matplotlib.patches as mpatches
        legend_patches = []

        for cls_name in sorted(class_counters.keys()):
            # 해당 클래스 요소가 1개 이상인 경우만 범례에 추가
            if class_counters[cls_name] > 0:
                # BGR -> RGB 변환
                b, g, r = class_colors[cls_name]
                rgb_color = (r/255, g/255, b/255)

                count = class_counters[cls_name]
                legend_patch = mpatches.Patch(color=rgb_color, label=f'{cls_name} ({count})')
                legend_patches.append(legend_patch)

        # 범례 표시 (많은 클래스가 있을 수 있으므로 작게 표시)
        if legend_patches:
            plt.legend(handles=legend_patches, loc='center left',
                      bbox_to_anchor=(1, 0.5), fontsize='small',
                      ncol=max(1, len(legend_patches)//20))

        plt.show()

        # 클래스별 통계 출력
        logger.info("\n감지된 클래스별 개수:")
        for cls, count in sorted(class_counters.items()):
            if count > 0:  # 감지된 항목만 표시
                logger.info(f"- {cls}: {count}개")

        return img_result

    except Exception as e:
        logger.error(f"결과 시각화 중 오류 발생: {e}")
        # 오류 발생 시 원본 이미지 반환
        return img

## 8. 전체 시스템 통합 함수

In [ ]:
def analyze_multiple_worksheets_advanced(image_list=None, model_choice="docstructbench", enable_resume=True):
    """고급 다중 학습지 분석 파이프라인 (배치 처리, 체크포인트, 메모리 최적화)"""
    start_time = time.time()
    logger.info(f"고급 다중 학습지 분석 시작 - 모델: {model_choice}")
    
    # 변수 초기화
    model = None
    all_results = []
    
    try:
        # 1. 체크포인트 복구 확인
        checkpoint_data = {}
        if enable_resume:
            checkpoint_data = checkpoint_manager.load_checkpoint()
            if checkpoint_data:
                user_choice = input("이전 진행률이 발견되었습니다. 이어서 진행하시겠습니까? (y/n): ")
                if user_choice.lower() == 'y':
                    logger.info("체크포인트에서 분석을 재개합니다.")
                else:
                    checkpoint_manager.clear_checkpoint()
                    checkpoint_data = {}
        
        # 2. 이미지 업로드 또는 로드
        if not image_list:
            image_list = upload_and_prepare_images()
            if not image_list:
                logger.error("이미지 업로드/로드 실패")
                return None
        
        # 3. 메모리 및 배치 크기 최적화
        stats = calculate_image_stats(image_list)
        if not check_memory_limits(stats["total_size_mb"]):
            user_choice = input("메모리 부족 경고가 있습니다. 계속 진행하시겠습니까? (y/n): ")
            if user_choice.lower() != 'y':
                logger.info("사용자가 처리를 중단했습니다.")
                return None
        
        # 동적 배치 크기 결정
        batch_size = adaptive_batch_size()
        logger.info(f"배치 크기: {batch_size}개 이미지")
        
        # 4. 모델 다운로드 및 로드 (한 번만)
        logger.info("모델 다운로드 및 로드 중...")
        try:
            model_path = download_model(model_choice)
            model, device = load_model(model_path)
        except Exception as e:
            logger.error(f"모델 로드 실패: {e}")
            return None
        
        # 5. 결과 디렉토리 설정
        output_dir = "analysis_results"
        if not ensure_directory(output_dir):
            logger.error("결과 디렉토리 생성 실패")
            return None
        
        # 6. 처리할 이미지 목록 결정 (체크포인트 고려)
        completed_images = set(checkpoint_data.get('completed', []))
        remaining_images = [img for i, img in enumerate(image_list) 
                           if f"{img['original_filename']}_page_{img['page_number']}" not in completed_images]
        
        if completed_images:
            logger.info(f"이미 완료된 이미지: {len(completed_images)}개")
            logger.info(f"남은 이미지: {len(remaining_images)}개")
        
        # 기존 결과 로드 (체크포인트가 있는 경우)
        if checkpoint_data.get('results'):
            all_results = checkpoint_data['results']
            logger.info(f"기존 결과 {len(all_results)}개 로드됨")
        
        # 7. 배치별 처리
        total_batches = (len(remaining_images) + batch_size - 1) // batch_size
        
        for batch_idx in range(total_batches):
            batch_start = batch_idx * batch_size
            batch_end = min(batch_start + batch_size, len(remaining_images))
            current_batch = remaining_images[batch_start:batch_end]
            
            logger.info(f"배치 {batch_idx + 1}/{total_batches} 처리 중 ({len(current_batch)}개 이미지)")
            
            # 메모리 상태 체크
            memory_status = memory_manager.check_memory_status()
            if memory_status == "critical":
                logger.error("메모리 부족으로 처리를 중단합니다.")
                break
            
            # 배치 처리 (병렬 또는 순차)
            batch_results = process_image_batch(current_batch, model, device, model_choice, output_dir)
            all_results.extend(batch_results)
            
            # 체크포인트 저장
            if enable_resume and (batch_idx + 1) % Config.CHECKPOINT_INTERVAL == 0:
                completed_list = [f"{r['original_filename']}_page_{r['page_number']}" 
                                for r in all_results if 'error' not in r]
                checkpoint_manager.save_checkpoint({
                    'completed': completed_list,
                    'results': all_results,
                    'batch_processed': batch_idx + 1,
                    'total_batches': total_batches
                })
            
            # 배치 완료 후 메모리 정리
            memory_manager.force_cleanup()
            
            logger.info(f"배치 {batch_idx + 1} 완료 - 누적 결과: {len(all_results)}개")
        
        # 8. 처리 완료 및 결과 정리
        total_time = time.time() - start_time
        successful_results = [r for r in all_results if 'error' not in r]
        failed_results = [r for r in all_results if 'error' in r]
        
        logger.success(f"고급 분석 완료 (소요 시간: {total_time:.2f}초)")
        logger.info(f"성공: {len(successful_results)}개, 실패: {len(failed_results)}개")
        
        # 9. 결과 요약 및 다운로드
        if successful_results:
            total_layouts = sum(r.get('detected_objects_count', 0) for r in successful_results)
            logger.info(f"총 {len(successful_results)}개 이미지에서 {total_layouts}개 레이아웃 영역 감지됨")
            
            # 환경별 결과 파일 처리
            if ENV == 'colab':
                download_choice = input("분석 결과 이미지를 다운로드하시겠습니까? (y/n): ")
                if download_choice.lower() == 'y':
                    try:
                        for result in successful_results:
                            if result.get('output_image_path') and files:
                                files.download(result['output_image_path'])
                        logger.info("✅ 모든 결과 파일 다운로드 완료!")
                    except Exception as e:
                        logger.error(f"다운로드 중 오류 발생: {e}")
            else:
                logger.info(f"분석 결과가 {output_dir} 폴더에 저장되었습니다.")
        
        # 10. 체크포인트 정리
        if enable_resume:
            checkpoint_manager.clear_checkpoint()
        
        return all_results if all_results else None
        
    except KeyboardInterrupt:
        logger.warning("사용자가 처리를 중단했습니다.")
        # 현재까지의 진행률 저장
        if enable_resume and all_results:
            completed_list = [f"{r['original_filename']}_page_{r['page_number']}" 
                            for r in all_results if 'error' not in r]
            checkpoint_manager.save_checkpoint({
                'completed': completed_list,
                'results': all_results,
                'interrupted': True
            })
            logger.info("진행률이 저장되었습니다. 다음에 이어서 처리할 수 있습니다.")
        return all_results if all_results else None
        
    except Exception as e:
        logger.error(f"고급 분석 중 심각한 오류 발생: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return None
        
    finally:
        # 리소스 정리
        if model is not None:
            try:
                # GPU 메모리 정리
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    torch.cuda.synchronize()
                del model
            except Exception as e:
                logger.warning(f"모델 정리 중 오류: {e}")
        
        # 메모리 정리
        memory_manager.force_cleanup()
        
        # 임시 파일 정리 (선택적)
        cleanup_choice = input("임시 파일을 정리하시겠습니까? (y/n): ")
        if cleanup_choice.lower() == 'y':
            temp_file_manager.cleanup_all()
        
        logger.info("리소스 정리 완료")

def process_image_batch(image_batch, model, device, model_choice, output_dir):
    """이미지 배치 처리 (병렬 지원)"""
    batch_results = []
    
    # 환경별 처리 방식 결정
    use_parallel = len(image_batch) > 1 and ENV == 'local'
    
    if use_parallel:
        # 병렬 처리 (로컬 환경에서만)
        logger.info(f"병렬 처리 시작 (워커 수: {Config.MAX_WORKERS})")
        
        with ThreadPoolExecutor(max_workers=Config.MAX_WORKERS) as executor:
            # 작업 제출
            future_to_img = {
                executor.submit(process_single_image_safe, img_info, model, device, model_choice, output_dir): img_info
                for img_info in image_batch
            }
            
            # 결과 수집
            for future in as_completed(future_to_img):
                img_info = future_to_img[future]
                try:
                    result = future.result()
                    if result:
                        batch_results.append(result)
                except Exception as e:
                    logger.error(f"병렬 처리 중 오류: {img_info['original_filename']} - {e}")
                    error_result = {
                        'original_filename': img_info['original_filename'],
                        'error': str(e),
                        'status': 'failed'
                    }
                    batch_results.append(error_result)
    else:
        # 순차 처리
        for img_info in tqdm(image_batch, desc="이미지 처리"):
            try:
                result = process_single_image_safe(img_info, model, device, model_choice, output_dir)
                if result:
                    batch_results.append(result)
            except Exception as e:
                logger.error(f"순차 처리 중 오류: {img_info['original_filename']} - {e}")
                error_result = {
                    'original_filename': img_info['original_filename'],
                    'error': str(e),
                    'status': 'failed'
                }
                batch_results.append(error_result)
    
    return batch_results

def process_single_image_safe(img_info, model, device, model_choice, output_dir):
    """단일 이미지 안전 처리 (메모리 관리 포함)"""
    try:
        logger.info(f"처리 중: {img_info['original_filename']} 페이지 {img_info['page_number']}")
        
        img = img_info['img_array']
        image_path = img_info['processed_filename']
        
        # 이미지 크기 확인
        height, width = img.shape[:2]
        logger.debug(f"이미지 크기: {width}x{height} 픽셀")
        
        # 레이아웃 분석
        layout_info = analyze_layout(model, image_path, img, device, model_choice)
        
        # TSPM용 별도 복사본 저장
        import copy
        layout_info_tspm = copy.deepcopy(layout_info)
        
        # 결과 시각화 (메모리 상태에 따라 조건부)
        memory_status = memory_manager.check_memory_status()
        if memory_status != "critical" and layout_info:
            try:
                result_img = visualize_results(img, layout_info, alpha=0.2)
            except Exception as e:
                logger.error(f"결과 시각화 중 오류: {e}")
                result_img = img.copy()
        else:
            result_img = img.copy()
        
        # 결과 저장
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        safe_name = safe_filename(img_info['original_filename'])
        
        if img_info['source_type'] == 'pdf':
            output_image_path = f"{output_dir}/{safe_name}_p{img_info['page_number']}_{timestamp}.jpg"
        else:
            output_image_path = f"{output_dir}/{safe_name}_{timestamp}.jpg"
        
        success = cv2.imwrite(output_image_path, result_img)
        if not success:
            logger.warning(f"결과 이미지 저장 실패: {output_image_path}")
            output_image_path = None
        
        # 결과 구성
        result = {
            'original_filename': img_info['original_filename'],
            'processed_filename': img_info['processed_filename'],
            'page_number': img_info.get('page_number', 1),
            'source_type': img_info['source_type'],
            'layout_info': layout_info,
            'layout_info_tspm': layout_info_tspm,
            'result_image': result_img,
            'output_image_path': output_image_path,
            'input_image_path': image_path,
            'processing_time': time.time(),
            'detected_objects_count': len(layout_info),
            'memory_usage_mb': memory_manager.get_memory_usage_mb()
        }
        
        logger.info(f"✅ {img_info['original_filename']} 페이지 {img_info['page_number']} 분석 완료 ({len(layout_info)}개 객체)")
        
        return result
        
    except Exception as e:
        logger.error(f"이미지 처리 실패: {img_info['original_filename']} - {e}")
        return {
            'original_filename': img_info['original_filename'],
            'error': str(e),
            'processing_time': time.time(),
            'status': 'failed'
        }

# 기존 함수 (호환성 유지하되 고급 버전 호출)
def analyze_multiple_worksheets(image_list=None, model_choice="docstructbench"):
    """환경 독립적 다중 학습지 분석 파이프라인 (호환성 유지)"""
    logger.info("기존 호환성 함수 호출됨 - 고급 버전으로 리다이렉트")
    return analyze_multiple_worksheets_advanced(image_list, model_choice, enable_resume=True)

# 기존 함수와의 호환성을 위한 wrapper
def analyze_worksheet(image_path=None, model_choice="docstructbench"):
    """단일 학습지 분석 (기존 함수 - 호환성 유지)"""
    try:
        if image_path:
            # 직접 경로가 주어진 경우
            img_info = process_image_file(image_path)
            if img_info:
                results = analyze_multiple_worksheets_advanced([img_info], model_choice, enable_resume=False)
                return results[0] if results and len(results) > 0 else None
        else:
            # 업로드 방식
            results = analyze_multiple_worksheets_advanced(None, model_choice, enable_resume=False)
            if results and len(results) == 1:
                return results[0]
            elif results and len(results) > 1:
                logger.info("여러 이미지가 업로드되었습니다. 첫 번째 결과를 반환합니다.")
                return results[0]
        return None
    except Exception as e:
        logger.error(f"단일 학습지 분석 중 오류: {e}")
        return None

# Colab 호환성을 위한 alias
analyze_multiple_worksheets_colab = analyze_multiple_worksheets_advanced

## 9. 분석 실행

In [ ]:
# ============================================
# 🚀 고급 환경 독립적 분석 실행 (전체 자동 처리)
# ============================================

# 환경 감지 (전역 변수 사용)
print(f"🚀 {ENV.upper()} 환경용 고급 다중 이미지 분석 시스템")
print("=" * 60)
print(f"💾 메모리 사용량: {memory_manager.get_memory_usage_mb():.1f}MB")
print(f"🖥️ 시스템 메모리: {memory_manager.get_memory_percent():.1%}")
print("=" * 60)

# 고급 기능 소개
print("🎯 새로운 기능:")
print("  ✅ 스트리밍 PDF 처리 (메모리 최적화)")
print("  ✅ 배치 처리 및 병렬 처리 지원")
print("  ✅ 실시간 메모리 모니터링")
print("  ✅ 자동 체크포인트 및 재시작")
print("  ✅ 자동 임시 파일 관리")
print("  ✅ 동적 배치 크기 조정")
print("  🆕 전체 이미지 자동 처리 (선택 불필요)")
print()

# 모델 선택 UI
print("사용할 모델을 선택하세요:")
print("1. DocStructBench (학습지 및 교과서에 최적화) - 권장")
print("2. DocLayNet-Docsynth300K (일반 문서에 최적화)")
print("3. DocSynth300K-pretrain (커스텀 미세조정용)")

user_choice = input("선택 (1-3, 기본값: 1): ").strip()
if user_choice == "2":
    model_choice = "doclaynet_docsynth"
elif user_choice == "3":
    model_choice = "docsynth300k"
    if ENV == 'colab':
        print("\n⚠️ 주의: DocSynth300K 사전훈련 모델은 Colab 환경에서 메모리 사용량이 높을 수 있습니다.")
else:
    model_choice = "docstructbench"

print(f"\n선택된 모델: {model_choice}")

# 처리 모드 선택
print("\n처리 모드를 선택하세요:")
print("1. 고급 모드 (배치 처리, 체크포인트, 메모리 최적화) - 권장")
print("2. 호환성 모드 (기존 방식)")

mode_choice = input("선택 (1-2, 기본값: 1): ").strip()
use_advanced_mode = mode_choice != "2"

if use_advanced_mode:
    print("🎯 고급 모드 선택됨")
    print(f"  - 배치 크기: {adaptive_batch_size()}개 (동적 조정)")
    if ENV == 'local':
        print(f"  - 병렬 워커: {Config.MAX_WORKERS}개")
    print(f"  - 체크포인트 간격: {Config.CHECKPOINT_INTERVAL}개 이미지마다")
    print(f"  - PDF 최적화: {Config.PDF_ZOOM_FACTOR}x zoom, {Config.PDF_MAX_PAGES_PER_BATCH}페이지 배치")
    print("  - 전체 이미지 자동 처리: 활성화")
else:
    print("🔄 호환성 모드 선택됨")
    print("  - 전체 이미지 자동 처리: 활성화")

print("\n📁 지원 파일 형식: JPG, PNG, PDF")

if ENV == 'colab':
    print("💡 Colab 최적화:")
    print(f"  - 메모리 경고 임계값: {Config.MEMORY_WARNING_THRESHOLD:.0%}")
    print(f"  - 배치 크기: {Config.BATCH_SIZE_COLAB}개 (기본)")
    print("📄 PDF는 스트리밍 방식으로 처리됩니다")
    print("🔄 여러 파일 선택 시 Ctrl+클릭 사용")
else:
    print("💡 로컬 환경 최적화:")
    print(f"  - 메모리 경고 임계값: {Config.MEMORY_WARNING_THRESHOLD:.0%}")
    print(f"  - 배치 크기: {Config.BATCH_SIZE_LOCAL}개 (기본)")
    print(f"  - 병렬 처리: {Config.MAX_WORKERS}개 워커")
    print("📄 PDF는 스트리밍 방식으로 처리됩니다")
    print("🔄 Ctrl+클릭으로 여러 파일 선택 가능")

print("\n🎯 **중요**: 모든 이미지가 자동으로 TSPM 처리됩니다. 별도 선택 불필요!")
print()

# 분석 실행
if use_advanced_mode:
    # 고급 모드 실행
    analysis_results = analyze_multiple_worksheets_advanced(
        image_list=None, 
        model_choice=model_choice, 
        enable_resume=True
    )
else:
    # 기존 호환성 모드
    analysis_results = analyze_multiple_worksheets(model_choice=model_choice)

# 결과 처리 및 전역 변수 설정 (모든 이미지 자동 처리)
if analysis_results:
    # 성공한 결과만 필터링
    successful_results = [r for r in analysis_results if 'error' not in r]
    
    if successful_results:
        # 결과 통계 출력
        total_images = len(successful_results)
        failed_images = len(analysis_results) - total_images
        total_objects = sum(r.get('detected_objects_count', 0) for r in successful_results)
        
        print(f"\n{'='*60}")
        print("📊 LAM 분석 결과 요약")
        print(f"{'='*60}")
        print(f"✅ 성공한 이미지: {total_images}개")
        if failed_images > 0:
            print(f"❌ 실패한 이미지: {failed_images}개")
        print(f"🎯 총 감지된 객체: {total_objects}개")
        print(f"💾 최종 메모리 사용량: {memory_manager.get_memory_usage_mb():.1f}MB")
        
        # 모든 이미지 정보 출력
        print(f"\n📋 처리된 이미지 목록:")
        for i, result in enumerate(successful_results):
            filename = result['original_filename']
            page_info = f" (페이지 {result['page_number']})" if result['source_type'] == 'pdf' else ""
            layout_count = len(result['layout_info_tspm'])
            memory_mb = result.get('memory_usage_mb', 0)
            print(f"  {i+1}. {filename}{page_info} - {layout_count}개 영역 ({memory_mb:.1f}MB)")
        
        # 전역 변수 설정 (모든 이미지 리스트와 호환성을 위한 첫 번째 이미지)
        all_analysis_results = analysis_results
        
        # 첫 번째 이미지를 기본값으로 설정 (기존 코드 호환성)
        first_result = successful_results[0]
        image_path = first_result['input_image_path']
        layout_info_tspm = first_result['layout_info_tspm']
        
        logger.info(f"✅ 전역 변수 설정 완료:")
        logger.info(f"   - all_analysis_results: {len(all_analysis_results)}개 이미지 결과")
        logger.info(f"   - image_path (첫 번째): {image_path}")
        logger.info(f"   - layout_info_tspm (첫 번째): {len(layout_info_tspm)}개 블록")
        
        # 모든 이미지 처리를 위한 추가 전역 변수
        all_images_for_tspm = []
        for result in successful_results:
            all_images_for_tspm.append({
                'image_path': result['input_image_path'],
                'layout_info_tspm': result['layout_info_tspm'],
                'original_filename': result['original_filename'],
                'page_number': result['page_number'],
                'source_type': result['source_type'],
                'result_data': result
            })
        
        logger.info(f"   - all_images_for_tspm: {len(all_images_for_tspm)}개 이미지 준비됨")
        
        # 간단한 layout_info_tspm 출력 (첫 번째 이미지만)
        print(f"\n✅ 첫 번째 이미지 layout_info_tspm 출력 (앞부분 요약):")
        print(f"   파일: {first_result['original_filename']}")
        if first_result['source_type'] == 'pdf':
            print(f"   페이지: {first_result['page_number']}")
        
        try:
            # 너무 길어지지 않게 앞 2개 항목만 요약 출력
            preview_data = layout_info_tspm[:2]
            print(json.dumps(preview_data, ensure_ascii=False, indent=2))
            if len(layout_info_tspm) > 2:
                print(f"\n⚠️ 총 {len(layout_info_tspm)}개 중 앞 2개만 출력했습니다.")
        except Exception as e:
            print(f"❌ layout_info_tspm 출력 중 오류 발생: {e}")
        
        print(f"\n🎯 자동 TSPM 처리 준비 완료!")
        print(f"   - 처리 대상: {len(all_images_for_tspm)}개 이미지")
        print(f"   - 현재 환경: {ENV.upper()}")
        print(f"   - 처리 방식: 전체 자동 처리")
        
        # 고급 모드 추가 정보
        if use_advanced_mode:
            print(f"   - 처리 모드: 고급 (배치 처리)")
            if failed_images == 0:
                print("   - 모든 이미지 성공적으로 처리됨 ✅")
            print(f"   - 임시 파일: {len(temp_file_manager.temp_files)}개 생성됨")
        
        print("\n🔄 다음 TSPM 셀들이 모든 이미지를 자동으로 처리합니다!")
        
    else:
        logger.error("❌ 성공적으로 처리된 이미지가 없습니다.")
        print("\n❌ 모든 이미지 처리가 실패했습니다.")
        print("   - 메모리 부족이나 파일 손상을 확인해주세요.")
        print("   - 더 작은 PDF나 이미지로 다시 시도해보세요.")
        
        # 실패 시 빈 리스트 설정
        all_analysis_results = []
        all_images_for_tspm = []
        
else:
    logger.error("❌ 분석 결과가 없습니다.")
    print("\n❌ 분석이 완료되지 않았습니다.")
    print("   - 파일을 다시 업로드해보세요.")
    print("   - 메모리 상태를 확인해보세요.")
    
    # 실패 시 빈 리스트 설정
    all_analysis_results = []
    all_images_for_tspm = []

# 시스템 상태 최종 체크
final_memory_status = memory_manager.check_memory_status()
print(f"\n🖥️ 최종 시스템 상태: {final_memory_status}")

if use_advanced_mode:
    print(f"📁 임시 파일 상태: {len(temp_file_manager.temp_files)}개 파일")
    if checkpoint_manager and os.path.exists(checkpoint_manager.checkpoint_file):
        print("💾 체크포인트: 정리됨")
    else:
        print("💾 체크포인트: 없음")

print("\n🚀 LAM 분석 완료 - TSPM 자동 처리 준비됨!")





# **TSPM (OpenAI Vision API)**

## **Colab 환경 설정 및 필요 라이브러리 설치 컴포넌트**

In [ ]:
# ============================================
# 📦 TSPM 환경 설정 확인 (통합 설치 후)
# ============================================

# 환경 감지 (전역 변수 사용)
print(f"🔧 TSPM 환경 설정 확인... (감지된 환경: {ENV})")

# 주요 라이브러리 설치 확인
try:
    import sys
    import cv2
    import pytesseract
    import numpy as np
    import matplotlib.pyplot as plt
    from tqdm.notebook import tqdm
    import requests
    from PIL import Image
    from loguru import logger
    
    print("✅ 모든 TSPM 필수 라이브러리 설치 확인 완료!")
    
except ImportError as e:
    print(f"❌ TSPM 라이브러리 import 실패: {e}")
    print("⚠️ LAM 섹션의 패키지 설치 셀을 먼저 실행해주세요.")

# 로그 설정 (TSPM 전용)
logger.remove()
logger.add(sys.stderr, level="INFO")
logger.add("tspm_processing.log", rotation="10 MB", level="DEBUG")

logger.info("✅ TSPM 환경 설정 완료!")

In [ ]:
# ============================================
# ✅ TSPM 라이브러리 확인 및 버전 체크
# ============================================

from loguru import logger

logger.info("🧪 TSPM 라이브러리 버전 체크...")

# 환경 정보 확인
logger.info(f"✅ 실행 환경: {ENV.upper()}")
logger.info(f"✅ Python 버전: {sys.version}")

# 주요 라이브러리 버전 확인
try:
    logger.info(f"✅ OpenCV 버전: {cv2.__version__}")
    logger.info(f"✅ numpy 버전: {np.__version__}")
    logger.info(f"✅ matplotlib 버전: {plt.matplotlib.__version__}")
    logger.info(f"✅ requests 버전: {requests.__version__}")
    logger.info(f"✅ Pillow 버전: {Image.__version__}")
    logger.info(f"✅ pytesseract 버전: {pytesseract.get_tesseract_version()}")
except Exception as e:
    logger.error(f"❌ 버전 확인 실패: {e}")

logger.info("✅ TSPM 라이브러리 버전 체크 완료!")

## **라이브러리 import 및 로그 설정 컴포넌트**

In [ ]:
# ============================================
# 📥 TSPM 라이브러리 import 및 로그 설정
# ============================================

# 🛠️ 시스템 및 파일 관리용
import os          # 파일 경로, 디렉토리 관리
import sys         # 시스템 경로, 출력 관리

# 🛠️ 이미지 처리 및 컴퓨터 비전
import cv2         # OpenCV: 이미지 읽기, 쓰기, 전처리
import numpy as np # Numpy: 이미지 배열 처리

# 🛠️ 시각화 및 그래프 출력
import matplotlib.pyplot as plt  # 이미지, 그래프 시각화

# 🛠️ OCR 및 이미지-텍스트 변환
import pytesseract               # Tesseract OCR Python 바인딩
from PIL import Image            # Python Imaging Library (Pillow)

# 🛠️ 데이터 인코딩 및 파일 처리
import json                      # JSON 파일 저장, 로드
import base64                    # 이미지 base64 인코딩 (API 전송용)
import io                        # 메모리 버퍼 처리 (파일 없이 데이터 저장)

# 🛠️ 시간 및 진행률 관리
import time                      # 타임스탬프, 처리 시간 기록
import textwrap                  # 텍스트 줄바꿈 처리
import re                        # 정규 표현식

# 🛠️ API 호출용 HTTP 클라이언트
import requests                  # REST API 요청

# 🛠️ 고급 로깅
from loguru import logger       # Logger 객체용

# 환경별 tqdm import
if ENV == 'colab':
    from tqdm.notebook import tqdm   # Colab/Jupyter용 예쁜 진행률 표시
else:
    from tqdm import tqdm           # 로컬 환경용 진행률 표시

# ============================================
# 🔧 TSPM 전용 로그 설정
# ============================================

# 기존 로그 핸들러 제거
logger.remove()

# 콘솔에 INFO 이상 로그 출력
logger.add(sys.stderr, level="INFO")

# DEBUG 이상 로그를 파일로 기록 (최대 10MB씩 회전)
logger.add("tspm_processing.log", rotation="10 MB", level="DEBUG")

# ✅ 확인 메시지
logger.info("✅ TSPM 라이브러리 import 및 로그 설정 완료!")

# ============================================
# 🧪 TSPM 라이브러리 호환성 확인
# ============================================

# LAM에서 정의된 Config 클래스 확인
if 'Config' in globals():
    logger.info("✅ LAM Config 클래스 사용 가능")
    logger.info(f"   - OCR 대상 클래스: {Config.OCR_TARGET_CLASSES}")
    logger.info(f"   - API 대상 클래스: {Config.API_TARGET_CLASSES}")
else:
    logger.warning("⚠️ LAM Config 클래스를 찾을 수 없습니다. LAM 섹션을 먼저 실행해주세요.")

# LAM에서 정의된 유틸리티 함수 확인
if 'safe_crop_image' in globals():
    logger.info("✅ LAM 유틸리티 함수 사용 가능")
else:
    logger.warning("⚠️ LAM 유틸리티 함수를 찾을 수 없습니다. LAM 섹션을 먼저 실행해주세요.")

logger.info("✅ TSPM 라이브러리 호환성 확인 완료!")

In [ ]:
# ============================================
# ✅ [셀2] 라이브러리 import 테스트 코드
# ============================================

from loguru import logger

logger.info("🧪 [테스트 시작] 라이브러리 기능 점검 중...")

# 1️⃣ OS 및 시스템 테스트
try:
    current_dir = os.getcwd()
    logger.info(f"✅ os 테스트: 현재 작업 디렉토리 → {current_dir}")
except Exception as e:
    logger.error(f"❌ os 테스트 실패: {e}")

# 2️⃣ OpenCV + Numpy 테스트
try:
    img = np.zeros((50, 50, 3), dtype=np.uint8)
    cv2.imwrite("test_opencv2.jpg", img)
    logger.info("✅ OpenCV + Numpy 테스트: 검정 이미지 생성 및 저장 성공")
except Exception as e:
    logger.error(f"❌ OpenCV + Numpy 테스트 실패: {e}")

# 3️⃣ Matplotlib 테스트
try:
    plt.imshow(img)
    plt.title("Matplotlib 테스트")
    plt.axis('off')
    plt.show()
    logger.info("✅ Matplotlib 테스트: 이미지 시각화 성공")
except Exception as e:
    logger.error(f"❌ Matplotlib 테스트 실패: {e}")

# 4️⃣ pytesseract + PIL 테스트
try:
    pil_img = Image.fromarray(img)
    text = pytesseract.image_to_string(pil_img, lang='eng')
    logger.info(f"✅ pytesseract 테스트: OCR 실행 성공 (추출 텍스트: '{text}')")
except Exception as e:
    logger.error(f"❌ pytesseract 테스트 실패: {e}")

# 5️⃣ json + base64 + io 테스트
try:
    buffer = io.BytesIO()
    pil_img.save(buffer, format="PNG")
    encoded_img = base64.b64encode(buffer.getvalue()).decode()
    json_data = json.dumps({"image_base64": encoded_img[:20] + "..."})
    logger.info(f"✅ json/base64/io 테스트: 인코딩 및 JSON 직렬화 성공")
except Exception as e:
    logger.error(f"❌ json/base64/io 테스트 실패: {e}")

# 6️⃣ requests 테스트
try:
    resp = requests.get("https://www.google.com")
    if resp.status_code == 200:
        logger.info("✅ requests 테스트: Google 접속 성공")
    else:
        logger.warning(f"⚠️ requests 테스트: 상태 코드 {resp.status_code}")
except Exception as e:
    logger.error(f"❌ requests 테스트 실패: {e}")

# 7️⃣ tqdm 테스트
try:
    for _ in tqdm(range(3), desc="tqdm 테스트"):
        pass
    logger.info("✅ tqdm 테스트: 진행률 표시 성공")
except Exception as e:
    logger.error(f"❌ tqdm 테스트 실패: {e}")

logger.info("🧪 [테스트 완료] Colab 셀 2 라이브러리 점검 종료!")


## **LAM 출력 데이터 로드 컴포넌트**

: LAM에서 필요한 데이터를 TSPM으로 불러오는 컴포넌트.

- Layout Analysis 의 결과 파이션 딕셔너리 값을 가진 layout_info 는 LAM 안에서 지속적으로 덮어쓰여짐. 따라서 Layout 분석 직후의 Key값을 백업변수인 layout_info_tspm 에 저장하고 전역변수화 해서 TSPM으로 불러옴.
- 분석 대상이 되는 학습지 이미지가 담긴 image_path 또한 전역변수화 해서 불러옴.
- 원본이미지를 OpenCV로 로드하여 출력하여 확인 가능함.

In [ ]:
# ============================================
# 📤 [셀3] LAM 출력 데이터 로드 컴포넌트 (전체 자동 처리 지원)
# ============================================

# ✅ LAM에서 넘어온 전체 이미지 분석 결과 불러오기
try:
    # 전체 이미지 처리용 전역 변수 확인
    if 'all_images_for_tspm' not in globals():
        logger.warning("⚠️ all_images_for_tspm 전역 변수가 없습니다. LAM 분석을 먼저 실행해주세요.")
        all_images_for_tspm = []
    else:
        assert isinstance(all_images_for_tspm, list), "all_images_for_tspm는 리스트여야 합니다."
        logger.info(f"✅ all_images_for_tspm 로드 성공! 총 {len(all_images_for_tspm)}개의 이미지 준비됨")
        
        # 각 이미지 정보 출력
        for i, img_info in enumerate(all_images_for_tspm):
            filename = img_info['original_filename']
            page_info = f" (페이지 {img_info['page_number']})" if img_info['source_type'] == 'pdf' else ""
            layout_count = len(img_info['layout_info_tspm'])
            logger.info(f"  {i+1}. {filename}{page_info} - {layout_count}개 레이아웃 블록")
            
except Exception as e:
    logger.error(f"❌ all_images_for_tspm 로드 실패: {e}")
    all_images_for_tspm = []

# ✅ 기존 호환성을 위한 단일 이미지 변수들도 유지
try:
    if 'all_analysis_results' not in globals():
        logger.warning("⚠️ all_analysis_results 전역 변수가 없습니다. LAM 분석을 먼저 실행해주세요.")
        all_analysis_results = []
    else:
        assert isinstance(all_analysis_results, list), "all_analysis_results는 리스트여야 합니다."
        logger.info(f"✅ all_analysis_results 로드 성공! 총 {len(all_analysis_results)}개의 이미지 분석 결과")
except Exception as e:
    logger.error(f"❌ all_analysis_results 로드 실패: {e}")
    all_analysis_results = []

try:
    if 'layout_info_tspm' not in globals():
        logger.warning("⚠️ layout_info_tspm 전역 변수가 없습니다. LAM 분석을 먼저 실행해주세요.")
        layout_info_tspm = []
    else:
        logger.info(f"✅ layout_info_tspm 로드 성공! 총 {len(layout_info_tspm)}개의 layout 블록 감지됨")
except Exception as e:
    logger.error(f"❌ layout_info_tspm 로드 실패: {e}")
    layout_info_tspm = []

try:
    if 'image_path' not in globals():
        logger.warning("⚠️ image_path 전역 변수가 없습니다. LAM 분석을 먼저 실행해주세요.")
        image_path = None
    else:
        logger.info(f"✅ image_path 로드 성공: {image_path}")
except Exception as e:
    logger.error(f"❌ image_path 로드 실패: {e}")
    image_path = None

# ✅ 모든 이미지 로드 및 검증
all_images_loaded = []
if all_images_for_tspm:
    for i, img_info in enumerate(all_images_for_tspm):
        try:
            img_path = img_info['image_path']
            if os.path.exists(img_path):
                img = cv2.imread(img_path)
                if img is not None:
                    height, width = img.shape[:2]
                    all_images_loaded.append({
                        'index': i,
                        'original_filename': img_info['original_filename'],
                        'page_number': img_info['page_number'],
                        'source_type': img_info['source_type'],
                        'img_path': img_path,
                        'img': img,
                        'layout_info': img_info['layout_info_tspm'],
                        'size': (width, height)
                    })
                    logger.info(f"✅ 이미지 {i+1} 로드 성공: {img_info['original_filename']} ({width}x{height})")
                else:
                    logger.error(f"❌ 이미지 {i+1} 로드 실패: {img_path}")
            else:
                logger.error(f"❌ 이미지 {i+1} 파일이 존재하지 않음: {img_path}")
        except Exception as e:
            logger.error(f"❌ 이미지 {i+1} 처리 중 오류: {e}")

# ✅ 기존 호환성을 위한 단일 이미지 변수 설정
if image_path and os.path.exists(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        height, width = img.shape[:2]
        logger.info(f"✅ 기본 이미지 로드 성공! 크기: {width}x{height} (W x H)")
    else:
        logger.error(f"❌ 기본 이미지 로드 실패: {image_path}")
        img = None
else:
    img = None

# ✅ 결과 요약
logger.info(f"✅ TSPM 전체 자동 처리 준비 완료!")
logger.info(f"   - 총 이미지 수: {len(all_images_loaded)}개")
logger.info(f"   - 기본 선택 이미지: {image_path}")
logger.info(f"   - 기본 layout 블록 수: {len(layout_info_tspm)}개")
logger.info(f"   - 전체 자동 처리 모드: 활성화")

# 처리 방식 안내
if len(all_images_loaded) > 1:
    logger.info("💡 모든 이미지가 자동으로 처리됩니다. 사용자 선택 불필요!")
    print(f"\n🎯 전체 자동 처리 모드")
    print(f"   - 대상 이미지: {len(all_images_loaded)}개")
    print(f"   - 처리 방식: 순차적 자동 처리")
    print(f"   - 사용자 개입: 불필요")
elif len(all_images_loaded) == 1:
    logger.info("💡 단일 이미지가 감지되었습니다. 자동으로 처리됩니다.")
    print(f"\n🎯 단일 이미지 자동 처리")
    print(f"   - 대상 이미지: 1개")
    print(f"   - 처리 방식: 자동 처리")
else:
    logger.warning("⚠️ 처리할 수 있는 이미지가 없습니다. LAM 분석을 먼저 실행해주세요.")
    print(f"\n❌ 처리할 이미지가 없습니다.")
    print(f"   - LAM 분석을 먼저 실행해주세요.")

print(f"\n🔄 다음 OCR 셀이 모든 이미지를 자동으로 처리합니다!")

## **OCR 컴포넌트**

### **Tesseract OCR 컴포넌트**

In [ ]:
# ============================================
# 📤 [셀4] OCR 컴포넌트 (다중 이미지 지원, 개선됨)
# ============================================

import pytesseract
from PIL import Image
import textwrap
import re
import matplotlib.pyplot as plt

# ============================================
# ✅ 전역 변수 확인
# ============================================
if 'all_images_loaded' not in globals():
    logger.warning("⚠️ all_images_loaded 변수가 없습니다. 이전 셀을 먼저 실행해주세요.")
    all_images_loaded = []

# 기존 단일 이미지 변수 확인 (호환성 유지)
if 'layout_info_tspm' not in globals() or 'img' not in globals():
    logger.warning("⚠️ layout_info_tspm 또는 img 변수가 없습니다. 단일 이미지 처리를 건너뜁니다.")
    single_image_available = False
else:
    single_image_available = True

# ============================================
# ✅ OCR 유틸리티 함수
# ============================================

def safe_crop_image(img, coordinates):
    """안전한 이미지 크롭 (경계 검사 포함)"""
    x1, y1, x2, y2 = coordinates
    img_height, img_width = img.shape[:2]
    
    # 좌표 보정
    x1 = max(0, min(x1, img_width - 1))
    y1 = max(0, min(y1, img_height - 1))
    x2 = max(x1 + 1, min(x2, img_width))
    y2 = max(y1 + 1, min(y2, img_height))
    
    return img[y1:y2, x1:x2]

def process_ocr_text(text, wrap_width=25):
    """OCR 텍스트 처리 및 포맷팅"""
    if not text or len(text.strip()) <= 1:
        return ""
    
    processed_lines = []
    for line in text.split('\n'):
        line = line.strip()
        if re.match(r"^\s*(\d+\.\s|[-*]\s)", line):
            # 번호나 목록 기호로 시작하는 줄은 그대로 유지
            processed_lines.append(line)
        else:
            # 일반 텍스트는 줄바꿈 처리
            wrapped = textwrap.fill(line, width=wrap_width)
            processed_lines.append(wrapped)
    
    return '\n'.join(processed_lines)

# ============================================
# ✅ 다중 이미지 OCR 처리
# ============================================
all_ocr_results = []  # 모든 이미지의 OCR 결과를 저장

if all_images_loaded:
    logger.info("🧪 다중 이미지 OCR 컴포넌트 실행 시작...")
    
    for img_data in all_images_loaded:
        img_idx = img_data['index']
        original_filename = img_data['original_filename']
        page_number = img_data['page_number']
        source_type = img_data['source_type']
        img = img_data['img']
        layout_info = img_data['layout_info']
        
        logger.info(f"📄 이미지 {img_idx + 1} OCR 처리 중: {original_filename}")
        if source_type == 'pdf':
            logger.info(f"   페이지 {page_number} 처리")
        
        # 이미지별 OCR 결과 저장
        image_ocr_results = []
        
        for layout in layout_info:
            cls_name = layout['class_name'].lower()
            
            if cls_name not in Config.OCR_TARGET_CLASSES:
                continue
            
            # 좌표 정보
            coordinates = layout['box']
            
            # 안전한 이미지 크롭
            try:
                cropped_img = safe_crop_image(img, coordinates)
                
                if cropped_img.size == 0:
                    logger.warning(f"크롭된 이미지가 비어있음: {original_filename} ID {layout['id']}")
                    continue
                
                # OCR 실행
                pil_img = Image.fromarray(cropped_img)
                text = pytesseract.image_to_string(
                    pil_img,
                    lang='kor',
                    config=Config.OCR_CONFIG
                ).strip()
                
                # 텍스트 처리
                processed_text = process_ocr_text(text)
                
                if processed_text:
                    ocr_result = {
                        'image_index': img_idx,
                        'original_filename': original_filename,
                        'page_number': page_number,
                        'source_type': source_type,
                        'id': layout['id'],
                        'class_name': cls_name,
                        'coordinates': coordinates,
                        'text': processed_text,
                        'raw_text': text,  # 원본 텍스트도 보존
                        'text_length': len(processed_text)
                    }
                    image_ocr_results.append(ocr_result)
                    logger.info(f"✅ OCR 완료: {original_filename} ID {layout['id']} / {cls_name} / {len(processed_text)}자")
                else:
                    logger.warning(f"⚠️ OCR 생략: {original_filename} ID {layout['id']} / {cls_name} (텍스트 없음)")
                    
            except Exception as e:
                logger.error(f"❌ OCR 실패: {original_filename} ID {layout['id']} / {cls_name} / 오류: {e}")
        
        # 전체 결과에 추가
        all_ocr_results.extend(image_ocr_results)
        logger.info(f"✅ 이미지 {img_idx + 1} OCR 완료: {len(image_ocr_results)}개 영역 처리")

# ============================================
# ✅ 단일 이미지 OCR 처리 (기존 호환성 유지)
# ============================================
ocr_results = []

if single_image_available and not all_images_loaded:
    logger.info("🧪 단일 이미지 OCR 컴포넌트 실행...")
    
    for layout in layout_info_tspm:
        cls_name = layout['class_name'].lower()
        
        if cls_name not in Config.OCR_TARGET_CLASSES:
            continue
        
        coordinates = layout['box']
        
        try:
            cropped_img = safe_crop_image(img, coordinates)
            
            if cropped_img.size == 0:
                logger.warning(f"크롭된 이미지가 비어있음: ID {layout['id']}")
                continue
            
            pil_img = Image.fromarray(cropped_img)
            text = pytesseract.image_to_string(
                pil_img,
                lang='kor',
                config=Config.OCR_CONFIG
            ).strip()
            
            processed_text = process_ocr_text(text)
            
            if processed_text:
                ocr_results.append({
                    'id': layout['id'],
                    'class_name': cls_name,
                    'coordinates': coordinates,
                    'text': processed_text,
                    'raw_text': text,
                    'text_length': len(processed_text)
                })
                logger.info(f"✅ OCR 완료: ID {layout['id']} / {cls_name} / {len(processed_text)}자")
                
        except Exception as e:
            logger.error(f"❌ OCR 실패: ID {layout['id']} / {cls_name} / 오류: {e}")

# ============================================
# ✅ 결과 출력 및 시각화
# ============================================
def display_ocr_result(result, img, wrap_width=25):
    """OCR 결과를 시각화하고 출력"""
    id = result['id']
    class_name = result['class_name']
    x1, y1, x2, y2 = result['coordinates']
    text = result['text']
    
    # 크롭된 이미지 생성
    cropped_img = safe_crop_image(img, [x1, y1, x2, y2])
    if cropped_img.size == 0:
        print(f"⚠️ ID {id}: 이미지 크롭 실패")
        return
    
    bordered_img = cv2.copyMakeBorder(
        cropped_img,
        top=5, bottom=5, left=5, right=5,
        borderType=cv2.BORDER_CONSTANT,
        value=[0, 0, 0]
    )
    cropped_img_rgb = cv2.cvtColor(bordered_img, cv2.COLOR_BGR2RGB)
    
    print(f"\n=========< ID {id} / {class_name.capitalize()} >==========")
    
    # 이미지 시각화
    plt.figure(figsize=(3, 2))
    plt.imshow(cropped_img_rgb)
    plt.axis('off')
    plt.title(f"ID {id} / {class_name.capitalize()}")
    plt.show()
    
    # 텍스트 출력
    print(text)
    print("=" * 40)

if all_ocr_results:
    logger.info("🖼️ 다중 이미지 OCR 결과 시각화 시작...")
    
    # 이미지별로 그룹화
    results_by_image = {}
    for result in all_ocr_results:
        img_key = f"{result['image_index']}_{result['original_filename']}"
        if img_key not in results_by_image:
            results_by_image[img_key] = []
        results_by_image[img_key].append(result)
    
    for img_key, results in results_by_image.items():
        # 이미지 정보
        first_result = results[0]
        original_filename = first_result['original_filename']
        page_number = first_result['page_number']
        source_type = first_result['source_type']
        img_idx = first_result['image_index']
        
        # 해당 이미지 로드
        img = all_images_loaded[img_idx]['img']
        
        print(f"\n{'='*60}")
        if source_type == 'pdf':
            print(f"📄 {original_filename} (페이지 {page_number}) OCR 결과")
        else:
            print(f"📄 {original_filename} OCR 결과")
        print(f"{'='*60}")
        
        for result in results:
            display_ocr_result(result, img)
    
    logger.info(f"✅ 다중 이미지 OCR 결과 총 {len(all_ocr_results)}개 항목 출력 완료!")
    
    # 전체 결과를 ocr_results에도 저장 (기존 코드 호환성)
    ocr_results = all_ocr_results

elif ocr_results:
    logger.info("🖼️ 단일 이미지 OCR 결과 시각화...")
    
    for result in ocr_results:
        display_ocr_result(result, img)
    
    logger.info(f"✅ 단일 이미지 OCR 결과 총 {len(ocr_results)}개 항목 출력 완료!")

else:
    logger.info("⚠️ 처리된 OCR 결과가 없습니다.")

logger.info(f"✅ OCR 컴포넌트 실행 완료! 총 {len(ocr_results)}개 영역 처리됨")

## **AI API 컴포넌트**

### **OpenAI GPT Vision API 컴포넌트**

In [ ]:
# ============================================
# 📤 [셀5] OpenAI GPT Vision API 컴포넌트 (다중 이미지 지원, 개선됨)
# ============================================

import cv2
import base64
import json
from PIL import Image
import io
import matplotlib.pyplot as plt
import textwrap
import re
import os

# OpenAI 클라이언트 import 및 설치 확인
try:
    import openai
except ImportError:
    logger.info("OpenAI 라이브러리 설치 중...")
    if ENV == 'colab':
        !pip install -q openai
    else:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "openai"])
    import openai

# ============================================
# 전역 변수 확인
# ============================================
if 'all_images_loaded' not in globals():
    logger.warning("⚠️ all_images_loaded 변수가 없습니다. 이전 셀을 먼저 실행해주세요.")
    all_images_loaded = []

# 기존 단일 이미지 변수 확인 (호환성 유지)
if 'layout_info_tspm' not in globals() or 'img' not in globals():
    logger.warning("⚠️ layout_info_tspm 또는 img 변수가 없습니다. 단일 이미지 처리를 건너뜁니다.")
    single_image_available = False
else:
    single_image_available = True

# ============================================
# OpenAI API 설정 (보안 개선)
# ============================================
def get_openai_api_key():
    """OpenAI API 키를 안전하게 가져오기"""
    # 환경 변수에서 먼저 시도
    api_key = os.getenv('OPENAI_API_KEY')
    if api_key:
        return api_key
    
    # 사용자 입력으로 받기
    if ENV == 'colab':
        from google.colab import userdata
        try:
            # Colab Secrets에서 시도
            api_key = userdata.get('OPENAI_API_KEY')
            if api_key:
                return api_key
        except:
            pass
    
    # 직접 입력 받기
    import getpass
    api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
    return api_key

# API 키 가져오기
api_key = get_openai_api_key()
if not api_key:
    logger.error("OpenAI API 키가 제공되지 않았습니다. API 컴포넌트를 건너뜁니다.")
    api_results = []
    all_api_results = []
else:
    try:
        client = openai.OpenAI(api_key=api_key)
        model = "gpt-4-turbo"
        logger.info("✅ OpenAI 클라이언트 초기화 완료")
    except Exception as e:
        logger.error(f"OpenAI 클라이언트 초기화 실패: {e}")
        api_results = []
        all_api_results = []
        client = None

# ============================================
# API 유틸리티 함수
# ============================================

def encode_image_to_base64(img_array):
    """이미지 배열을 base64로 인코딩"""
    try:
        pil_img = Image.fromarray(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
        buffered = io.BytesIO()
        pil_img.save(buffered, format="PNG")
        return base64.b64encode(buffered.getvalue()).decode("utf-8")
    except Exception as e:
        logger.error(f"이미지 인코딩 실패: {e}")
        return None

def create_vision_prompt(cls_name):
    """클래스별 최적화된 프롬프트 생성"""
    base_prompts = {
        'figure': "이 그림이나 도표의 내용을 시각 장애 아동이 이해할 수 있도록 명확하고 구체적으로 설명해 주세요.",
        'table': "이 표의 구조와 내용을 시각 장애 아동이 이해할 수 있도록 체계적으로 설명해 주세요."
    }
    
    return base_prompts.get(cls_name, f"이 {cls_name}의 내용을 간단히 설명해 주세요.")

# ============================================
# ✅ 다중 이미지 API 처리
# ============================================
all_api_results = []  # 모든 이미지의 API 결과를 저장

if all_images_loaded and client:
    logger.info("🤖 다중 이미지 OpenAI GPT Vision API 컴포넌트 실행 시작...")
    
    for img_data in all_images_loaded:
        img_idx = img_data['index']
        original_filename = img_data['original_filename']
        page_number = img_data['page_number']
        source_type = img_data['source_type']
        img = img_data['img']
        layout_info = img_data['layout_info']
        
        logger.info(f"📄 이미지 {img_idx + 1} API 처리 중: {original_filename}")
        if source_type == 'pdf':
            logger.info(f"   페이지 {page_number} 처리")
        
        # 이미지별 API 결과 저장
        image_api_results = []
        
        for layout in layout_info:
            cls_name = layout['class_name'].lower()
            if cls_name not in Config.API_TARGET_CLASSES:
                continue
            
            coordinates = layout['box']
            
            # 안전한 이미지 크롭 (OCR 컴포넌트의 함수 재사용)
            try:
                cropped_img = safe_crop_image(img, coordinates)
                
                if cropped_img.size == 0:
                    logger.warning(f"크롭된 이미지가 비어있음: {original_filename} ID {layout['id']}")
                    continue
                
                # 이미지를 base64로 인코딩
                img_base64 = encode_image_to_base64(cropped_img)
                if not img_base64:
                    logger.error(f"이미지 인코딩 실패: {original_filename} ID {layout['id']}")
                    continue
                
                # 프롬프트 생성
                prompt = create_vision_prompt(cls_name)
                
                # OpenAI API 호출
                response = client.chat.completions.create(
                    model=model,
                    messages = [
                        {
                            "role": "system",
                            "content": (
                                "**당신은 시각 장애 아동을 위한 국어, 수학, 영어 학습 AI 비서입니다.**\n\n"
                                "당신의 역할은 초등학생 시각 장애 아동이 그림, 사진, 표, 그래프 등이 포함된 학습 자료를 명확하게 이해할 수 있도록 시각 정보를 텍스트로 설명하는 것입니다. "
                                "모든 설명은 **최신 한국 점자 규정(국립국어원 고시)**을 준수해야 하며, 아이들의 눈높이에 맞춰 간결하고 이해하기 쉽게 작성되어야 합니다.\n\n"
                                "---\n\n"
                                "**[주요 임무]**\n\n"
                                "사용자가 제공하는 시각 자료(그림, 사진, 표, 그래프 등)의 내용을 한국어로 간결하고 핵심만 요약하여 설명합니다. 설명은 음성으로 변환되었을 때 명확하게 전달될 수 있도록 작성해야 합니다.\n\n"
                                "**[작성 원칙]**\n\n"
                                "1.  **직접적이고 명료한 설명:** 불필요한 수식어나 비유적 표현을 피하고, 시각적 정보를 있는 그대로 묘사합니다. 문장은 짧고 명확하게 구성합니다.\n"
                                "2.  **핵심 정보 강조:** 각 과목의 학습 목표와 문제 해결에 필수적인 정보를 우선적으로 전달합니다.\n"
                                "3.  **논리적 순서:** 정보는 이해하기 쉬운 논리적인 순서에 따라 설명합니다.\n"
                                "    *   **전체 설명:** 자료의 종류(예: 이야기 그림, 막대그래프, 단어 카드)와 전체적인 주제를 먼저 언급합니다.\n"
                                "    *   **부분 설명:** 중요한 부분부터 세부적인 내용으로 순차적으로 설명합니다.\n"
                                "4.  **한국 점자 규정 준수:**\n"
                                "    *   국어, 수학, 영어의 기호, 단위, 약어 등은 **'개정 한국 점자 규정 해설서'** 의 표기법을 따릅니다.\n"
                                "    *   특히 영어의 경우 로마자 표기법을, 수학의 경우 숫자 및 연산 기호 표기법을 정확히 준수합니다.\n"
                                "5.  **대체 텍스트 형식 활용:**\n"
                                "    *   설명 시작과 끝을 명확히 하여 정보의 범위를 알려줍니다. 예: \"[그림 설명 시작]...[그림 설명 끝]\"\n"
                                "    *   이해를 돕기 위한 부가 설명은 '점역자 주' 형식에 따라 괄호 안에 제공할 수 있습니다. 예: (점역자 주: 이 그림은 두 사람이 손을 잡고 있는 행복한 모습입니다.)\n\n"
                                "**[과목별 작성 지침 및 예시]**\n\n"
                                "*   **국어**\n"
                                "    *   **지침:** 이야기 그림은 등장인물, 배경, 핵심 사건 순서로 설명합니다. 인물의 표정이나 행동을 객관적으로 묘사하여 감정을 유추할 수 있도록 돕습니다.\n"
                                "    *   **예시 (흥부전 삽화):**\n"
                                "        > [그림 설명 시작]\n"
                                "        > 흥부전 이야기 그림입니다.\n"
                                "        > 제비 한 마리가 다리를 다친 채 마당에 떨어져 있습니다.\n"
                                "        > 흥부와 아내가 걱정스러운 표정으로 제비를 살펴보고 있습니다.\n"
                                "        > [그림 설명 끝]\n\n"
                                "*   **수학**\n"
                                "    *   **지침:** 도형은 전체적인 모양을 먼저 설명하고 각 부분의 명칭, 길이, 각도 등 수치 정보를 정확히 전달합니다. 그래프나 표는 제목, 가로축과 세로축의 의미, 각 항목의 값과 변화 추세를 순서대로 설명합니다.\n"
                                "    *   **예시 (막대그래프):**\n"
                                "        > [그래프 설명 시작]\n"
                                "        > '좋아하는 과일' 막대그래프입니다.\n"
                                "        > 가로축은 과일 종류, 세로축은 학생 수입니다.\n"
                                "        > 사과는 5명, 바나나는 8명, 딸기는 3명입니다.\n"
                                "        > (점역자 주: 바나나를 좋아하는 학생 수가 가장 많습니다.)\n"
                                "        > [그래프 설명 끝]\n\n"
                                "*   **영어**\n"
                                "    *   **지침:** 그림과 함께 제시된 알파벳이나 단어를 명확히 연결하여 설명합니다. 그림을 묘사한 후, 관련된 영어 단어와 그 철자를 정확하게 알려줍니다.\n"
                                "    *   **예시 (알파벳 카드):**\n"
                                "        > [그림 설명 시작]\n"
                                "        > 사과 그림이 있는 알파벳 카드입니다.\n"
                                "        > 그림 옆에 'apple'이라는 단어가 쓰여있습니다.\n"
                                "        > 철자는 a, p, p, l, e 입니다.\n"
                                "        > [그림 설명 끝]"
                            )
                        },
                        {
                            "role": "user",
                            "content": [
                                {"type": "text", "text": prompt},
                                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_base64}"}}
                            ]
                        }
                    ],
                    
                    temperature=0.2,
                    top_p=0.95,
                    max_tokens=600,
                    frequency_penalty=0.5,
                    presence_penalty=0.0,
                    seed=42
                )
                
                description = response.choices[0].message.content.strip()
                
                api_result = {
                    'image_index': img_idx,
                    'original_filename': original_filename,
                    'page_number': page_number,
                    'source_type': source_type,
                    'id': layout['id'],
                    'class_name': cls_name,
                    'coordinates': coordinates,
                    'description': description,
                    'description_length': len(description)
                }
                image_api_results.append(api_result)
                
                logger.info(f"✅ API 응답 완료: {original_filename} ID {layout['id']} / {cls_name} → '{description[:30]}...'")
                
            except Exception as e:
                logger.error(f"❌ API 요청 실패: {original_filename} ID {layout['id']} / {cls_name} → 오류: {e}")
        
        # 전체 결과에 추가
        all_api_results.extend(image_api_results)
        logger.info(f"✅ 이미지 {img_idx + 1} API 완료: {len(image_api_results)}개 영역 처리")

# ============================================
# ✅ 단일 이미지 API 처리 (기존 호환성 유지)
# ============================================
api_results = []

if single_image_available and not all_images_loaded and client:
    logger.info("🤖 단일 이미지 OpenAI GPT Vision API 컴포넌트 실행...")
    
    for layout in layout_info_tspm:
        cls_name = layout['class_name'].lower()
        if cls_name not in Config.API_TARGET_CLASSES:
            continue
        
        coordinates = layout['box']
        
        try:
            cropped_img = safe_crop_image(img, coordinates)
            
            if cropped_img.size == 0:
                logger.warning(f"크롭된 이미지가 비어있음: ID {layout['id']}")
                continue
            
            img_base64 = encode_image_to_base64(cropped_img)
            if not img_base64:
                logger.error(f"이미지 인코딩 실패: ID {layout['id']}")
                continue
            
            prompt = create_vision_prompt(cls_name)
            
            response = client.chat.completions.create(
                model=model,
                messages = [
                    {
                        "role": "system",
                        "content": (
                            "**당신은 시각 장애 아동을 위한 국어, 수학, 영어 학습 AI 비서입니다.**\n\n"
                            "당신의 역할은 초등학생 시각 장애 아동이 그림, 사진, 표, 그래프 등이 포함된 학습 자료를 명확하게 이해할 수 있도록 시각 정보를 텍스트로 설명하는 것입니다. "
                            "모든 설명은 **최신 한국 점자 규정(국립국어원 고시)**을 준수해야 하며, 아이들의 눈높이에 맞춰 간결하고 이해하기 쉽게 작성되어야 합니다.\n\n"
                            "---\n\n"
                            "**[주요 임무]**\n\n"
                            "사용자가 제공하는 시각 자료(그림, 사진, 표, 그래프 등)의 내용을 한국어로 간결하고 핵심만 요약하여 설명합니다. 설명은 음성으로 변환되었을 때 명확하게 전달될 수 있도록 작성해야 합니다.\n\n"
                            "**[작성 원칙]**\n\n"
                            "1.  **직접적이고 명료한 설명:** 불필요한 수식어나 비유적 표현을 피하고, 시각적 정보를 있는 그대로 묘사합니다. 문장은 짧고 명확하게 구성합니다.\n"
                            "2.  **핵심 정보 강조:** 각 과목의 학습 목표와 문제 해결에 필수적인 정보를 우선적으로 전달합니다.\n"
                            "3.  **논리적 순서:** 정보는 이해하기 쉬운 논리적인 순서에 따라 설명합니다.\n"
                            "    *   **전체 설명:** 자료의 종류(예: 이야기 그림, 막대그래프, 단어 카드)와 전체적인 주제를 먼저 언급합니다.\n"
                            "    *   **부분 설명:** 중요한 부분부터 세부적인 내용으로 순차적으로 설명합니다.\n"
                            "4.  **한국 점자 규정 준수:**\n"
                            "    *   국어, 수학, 영어의 기호, 단위, 약어 등은 **'개정 한국 점자 규정 해설서'** 의 표기법을 따릅니다.\n"
                            "    *   특히 영어의 경우 로마자 표기법을, 수학의 경우 숫자 및 연산 기호 표기법을 정확히 준수합니다.\n"
                            "5.  **대체 텍스트 형식 활용:**\n"
                            "    *   설명 시작과 끝을 명확히 하여 정보의 범위를 알려줍니다. 예: \"[그림 설명 시작]...[그림 설명 끝]\"\n"
                            "    *   이해를 돕기 위한 부가 설명은 '점역자 주' 형식에 따라 괄호 안에 제공할 수 있습니다. 예: (점역자 주: 이 그림은 두 사람이 손을 잡고 있는 행복한 모습입니다.)\n\n"
                            "**[과목별 작성 지침 및 예시]**\n\n"
                            "*   **국어**\n"
                            "    *   **지침:** 이야기 그림은 등장인물, 배경, 핵심 사건 순서로 설명합니다. 인물의 표정이나 행동을 객관적으로 묘사하여 감정을 유추할 수 있도록 돕습니다.\n"
                            "    *   **예시 (흥부전 삽화):**\n"
                            "        > [그림 설명 시작]\n"
                            "        > 흥부전 이야기 그림입니다.\n"
                            "        > 제비 한 마리가 다리를 다친 채 마당에 떨어져 있습니다.\n"
                            "        > 흥부와 아내가 걱정스러운 표정으로 제비를 살펴보고 있습니다.\n"
                            "        > [그림 설명 끝]\n\n"
                            "*   **수학**\n"
                            "    *   **지침:** 도형은 전체적인 모양을 먼저 설명하고 각 부분의 명칭, 길이, 각도 등 수치 정보를 정확히 전달합니다. 그래프나 표는 제목, 가로축과 세로축의 의미, 각 항목의 값과 변화 추세를 순서대로 설명합니다.\n"
                            "    *   **예시 (막대그래프):**\n"
                            "        > [그래프 설명 시작]\n"
                            "        > '좋아하는 과일' 막대그래프입니다.\n"
                            "        > 가로축은 과일 종류, 세로축은 학생 수입니다.\n"
                            "        > 사과는 5명, 바나나는 8명, 딸기는 3명입니다.\n"
                            "        > (점역자 주: 바나나를 좋아하는 학생 수가 가장 많습니다.)\n"
                            "        > [그래프 설명 끝]\n\n"
                            "*   **영어**\n"
                            "    *   **지침:** 그림과 함께 제시된 알파벳이나 단어를 명확히 연결하여 설명합니다. 그림을 묘사한 후, 관련된 영어 단어와 그 철자를 정확하게 알려줍니다.\n"
                            "    *   **예시 (알파벳 카드):**\n"
                            "        > [그림 설명 시작]\n"
                            "        > 사과 그림이 있는 알파벳 카드입니다.\n"
                            "        > 그림 옆에 'apple'이라는 단어가 쓰여있습니다.\n"
                            "        > 철자는 a, p, p, l, e 입니다.\n"
                            "        > [그림 설명 끝]"
                        )
                    },
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_base64}"}}
                        ]
                    }
                ],
                
                temperature=0.2,
                top_p=0.95,
                max_tokens=600,
                frequency_penalty=0.5,
                presence_penalty=0.0,
                seed=42
            )
            
            description = response.choices[0].message.content.strip()
            
            api_results.append({
                'id': layout['id'],
                'class_name': cls_name,
                'coordinates': coordinates,
                'description': description,
                'description_length': len(description)
            })
            
            logger.info(f"✅ API 응답 완료: ID {layout['id']} / {cls_name} → '{description[:30]}...'")
            
        except Exception as e:
            logger.error(f"❌ API 요청 실패: ID {layout['id']} / {cls_name} → 오류: {e}")

# ============================================
# ✅ 결과 출력 및 시각화 (OCR과 동일한 방식 사용)
# ============================================
def display_api_result(result, img, wrap_width=25):
    """API 결과를 시각화하고 출력"""
    id = result['id']
    class_name = result['class_name']
    x1, y1, x2, y2 = result['coordinates']
    description = result['description']
    
    # 크롭된 이미지 생성
    cropped_img = safe_crop_image(img, [x1, y1, x2, y2])
    if cropped_img.size == 0:
        print(f"⚠️ ID {id}: 이미지 크롭 실패")
        return
    
    cropped_img_rgb = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
    
    print(f"\n============< ID {id} / {class_name.capitalize()} >=============")
    
    # 이미지 시각화
    plt.figure(figsize=(3, 2))
    plt.imshow(cropped_img_rgb)
    plt.axis('off')
    plt.title(f"ID {id} / {class_name.capitalize()}")
    plt.show()
    
    # 설명 줄바꿈 처리
    processed_lines = []
    for line in description.split('\n'):
        line = line.strip()
        if re.match(r"^\s*(\d+\.\s|[-*]\s)", line):
            processed_lines.append(line)
        else:
            wrapped = textwrap.fill(line, width=wrap_width)
            processed_lines.append(wrapped)
    
    final_description = '\n'.join(processed_lines)
    print(final_description)
    print("=" * 40)

if all_api_results:
    logger.info("🖼️ 다중 이미지 API 결과 시각화 시작...")
    
    # 이미지별로 그룹화
    results_by_image = {}
    for result in all_api_results:
        img_key = f"{result['image_index']}_{result['original_filename']}"
        if img_key not in results_by_image:
            results_by_image[img_key] = []
        results_by_image[img_key].append(result)
    
    for img_key, results in results_by_image.items():
        # 이미지 정보
        first_result = results[0]
        original_filename = first_result['original_filename']
        page_number = first_result['page_number']
        source_type = first_result['source_type']
        img_idx = first_result['image_index']
        
        # 해당 이미지 로드
        img = all_images_loaded[img_idx]['img']
        
        print(f"\n{'='*60}")
        if source_type == 'pdf':
            print(f"🤖 {original_filename} (페이지 {page_number}) API 결과")
        else:
            print(f"🤖 {original_filename} API 결과")
        print(f"{'='*60}")
        
        for result in results:
            display_api_result(result, img)
    
    logger.info("✅ 다중 이미지 API 결과 시각화 및 출력 완료!")
    
    # 전체 결과를 api_results에도 저장 (기존 코드 호환성)
    api_results = all_api_results

elif api_results:
    logger.info("🖼️ 단일 이미지 API 결과 시각화...")
    
    for result in api_results:
        display_api_result(result, img)
    
    logger.info("✅ 단일 이미지 API 결과 시각화 및 출력 완료!")

else:
    logger.info("⚠️ 처리된 API 결과가 없습니다.")

logger.info(f"🤖 OpenAI GPT Vision API 컴포넌트 완료! 총 {len(api_results)}개 영역 처리됨")

# **CIM**

### **OCR 결과 & AI API 결과 객체 출력 컴포넌트**

In [ ]:
import pprint

# ============================================
# 전역 변수 확인 (다중 이미지 지원)
# ============================================
if 'ocr_results' not in globals():
    logger.warning("⚠️ ocr_results 변수가 없습니다. OCR 컴포넌트를 먼저 실행해주세요.")
    ocr_results = []

if 'api_results' not in globals():
    logger.warning("⚠️ api_results 변수가 없습니다. API 컴포넌트를 먼저 실행해주세요.")
    api_results = []

# ============================================
# 📝 다중 이미지 대응 OCR 결과 리스트 전체 출력
# ============================================

logger.info("📝 OCR 결과 리스트 전체 출력 시작...")

pp = pprint.PrettyPrinter(indent=2, width=120)

if len(ocr_results) > 0:
    # 이미지별로 그룹화하여 출력
    if any('image_index' in item for item in ocr_results):
        # 다중 이미지 결과인 경우
        logger.info("📄 다중 이미지 OCR 결과 출력:")
        
        # 이미지별로 그룹화
        results_by_image = {}
        for item in ocr_results:
            if 'image_index' in item:
                img_key = f"{item['image_index']}_{item['original_filename']}"
                if img_key not in results_by_image:
                    results_by_image[img_key] = []
                results_by_image[img_key].append(item)
            else:
                # 기존 단일 이미지 형식
                img_key = "single_image"
                if img_key not in results_by_image:
                    results_by_image[img_key] = []
                results_by_image[img_key].append(item)
        
        for img_key, img_results in results_by_image.items():
            if img_key != "single_image":
                first_item = img_results[0]
                filename = first_item['original_filename']
                page_number = first_item.get('page_number', 1)
                source_type = first_item.get('source_type', 'image')
                
                print(f"\n{'='*80}")
                if source_type == 'pdf':
                    print(f"📄 {filename} (페이지 {page_number}) OCR 결과 ({len(img_results)}개 항목)")
                else:
                    print(f"📄 {filename} OCR 결과 ({len(img_results)}개 항목)")
                print(f"{'='*80}")
            else:
                print(f"\n{'='*80}")
                print(f"📄 단일 이미지 OCR 결과 ({len(img_results)}개 항목)")
                print(f"{'='*80}")
            
            for item in img_results:
                pp.pprint(item)
                print("-" * 120)  # 항목별 구분선
    else:
        # 단일 이미지 결과인 경우
        logger.info("📄 단일 이미지 OCR 결과 출력:")
        for item in ocr_results:
            pp.pprint(item)
            print("=" * 120)  # 구분선
    
    logger.info(f"✅ OCR 결과 총 {len(ocr_results)}개 항목 출력 완료!")
else:
    logger.info("OCR 결과가 없습니다.")

# ============================================
# 📝 다중 이미지 대응 API 결과 리스트 전체 출력
# ============================================

logger.info("📝 API 결과 리스트 전체 출력 시작...")

if len(api_results) > 0:
    # 이미지별로 그룹화하여 출력
    if any('image_index' in item for item in api_results):
        # 다중 이미지 결과인 경우
        logger.info("🤖 다중 이미지 API 결과 출력:")
        
        # 이미지별로 그룹화
        results_by_image = {}
        for item in api_results:
            if 'image_index' in item:
                img_key = f"{item['image_index']}_{item['original_filename']}"
                if img_key not in results_by_image:
                    results_by_image[img_key] = []
                results_by_image[img_key].append(item)
            else:
                # 기존 단일 이미지 형식
                img_key = "single_image"
                if img_key not in results_by_image:
                    results_by_image[img_key] = []
                results_by_image[img_key].append(item)
        
        for img_key, img_results in results_by_image.items():
            if img_key != "single_image":
                first_item = img_results[0]
                filename = first_item['original_filename']
                page_number = first_item.get('page_number', 1)
                source_type = first_item.get('source_type', 'image')
                
                print(f"\n{'='*80}")
                if source_type == 'pdf':
                    print(f"🤖 {filename} (페이지 {page_number}) API 결과 ({len(img_results)}개 항목)")
                else:
                    print(f"🤖 {filename} API 결과 ({len(img_results)}개 항목)")
                print(f"{'='*80}")
            else:
                print(f"\n{'='*80}")
                print(f"🤖 단일 이미지 API 결과 ({len(img_results)}개 항목)")
                print(f"{'='*80}")
            
            for item in img_results:
                pp.pprint(item)
                print("-" * 120)  # 항목별 구분선
    else:
        # 단일 이미지 결과인 경우
        logger.info("🤖 단일 이미지 API 결과 출력:")
        for item in api_results:
            pp.pprint(item)
            print("=" * 120)  # 구분선
    
    logger.info(f"✅ API 결과 총 {len(api_results)}개 항목 출력 완료!")
else:
    logger.info("API 결과가 없습니다.")

# ============================================
# 📊 다중 이미지 대응 객체 카운터
# ============================================

logger.info("📊 객체 카운터 실행 시작...")

from collections import Counter

# OCR 쪽 카운트 (다중 이미지 대응)
if any('image_index' in item for item in ocr_results):
    # 다중 이미지 결과인 경우
    ocr_class_counts = Counter(item['class_name'] for item in ocr_results)
    ocr_image_counts = Counter()
    
    for item in ocr_results:
        if 'original_filename' in item:
            filename = item['original_filename']
            if item.get('source_type') == 'pdf':
                key = f"{filename} (페이지 {item['page_number']})"
            else:
                key = filename
            ocr_image_counts[key] += 1
        else:
            ocr_image_counts['단일 이미지'] += 1
else:
    # 단일 이미지 결과인 경우
    ocr_class_counts = Counter(item['class_name'] for item in ocr_results)
    ocr_image_counts = Counter({'단일 이미지': len(ocr_results)})

ocr_total = len(ocr_results)

# API 쪽 카운트 (다중 이미지 대응)
if any('image_index' in item for item in api_results):
    # 다중 이미지 결과인 경우
    api_class_counts = Counter(item['class_name'] for item in api_results)
    api_image_counts = Counter()
    
    for item in api_results:
        if 'original_filename' in item:
            filename = item['original_filename']
            if item.get('source_type') == 'pdf':
                key = f"{filename} (페이지 {item['page_number']})"
            else:
                key = filename
            api_image_counts[key] += 1
        else:
            api_image_counts['단일 이미지'] += 1
else:
    # 단일 이미지 결과인 경우
    api_class_counts = Counter(item['class_name'] for item in api_results)
    api_image_counts = Counter({'단일 이미지': len(api_results)})

api_total = len(api_results)

# 전체 합계
total_objects = ocr_total + api_total

# ✅ 출력
logger.info(f"✅ 탐지된 총 객체 수 (OCR + API): {total_objects}")

# 이미지별 통계
print(f"\n{'='*60}")
print("📊 이미지별 처리 통계")
print(f"{'='*60}")

# OCR 이미지별 통계
if ocr_image_counts:
    print(f"\n📝 OCR 처리 이미지별 개수:")
    for img_name, count in ocr_image_counts.most_common():
        print(f"   - {img_name}: {count}개")

# API 이미지별 통계
if api_image_counts:
    print(f"\n🤖 API 처리 이미지별 개수:")
    for img_name, count in api_image_counts.most_common():
        print(f"   - {img_name}: {count}개")

# 클래스별 통계
print(f"\n📝 OCR 총 {ocr_total}개 (클래스별 개수):")
for cls, count in ocr_class_counts.most_common():
    print(f"   - {cls}: {count}개")

print(f"\n🤖 API 총 {api_total}개 (클래스별 개수):")
for cls, count in api_class_counts.most_common():
    print(f"   - {cls}: {count}개")

# 전체 클래스 통계
all_classes = set(ocr_class_counts.keys()) | set(api_class_counts.keys())
if all_classes:
    print(f"\n🎯 전체 처리된 클래스 유형 ({len(all_classes)}개):")
    for cls in sorted(all_classes):
        ocr_count = ocr_class_counts.get(cls, 0)
        api_count = api_class_counts.get(cls, 0)
        total_cls_count = ocr_count + api_count
        print(f"   - {cls}: {total_cls_count}개 (OCR: {ocr_count}, API: {api_count})")

logger.info("✅ 객체 카운터 실행 완료!")

### **정렬 & 문제 그룹화 컴포넌트**

### **1차 가공 문서 시각화 컴포넌트**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

# ============================================
# 전역 변수 확인 (다중 이미지 지원)
# ============================================
if 'all_images_loaded' not in globals():
    logger.warning("⚠️ all_images_loaded 변수가 없습니다. 단일 이미지 처리로 진행합니다.")
    all_images_loaded = []

# 기존 단일 이미지 변수 확인 (호환성 유지)
if 'img' not in globals():
    logger.warning("⚠️ img 변수가 없습니다.")
    img = None

if 'ocr_results' not in globals():
    logger.warning("⚠️ ocr_results 변수가 없습니다.")
    ocr_results = []

if 'api_results' not in globals():
    logger.warning("⚠️ api_results 변수가 없습니다.")
    api_results = []

# ============================================
# 🔧 유틸 함수: 픽셀 기반 줄바꿈
# ============================================

def wrap_text_to_box(text, font, box_width):
    wrapped_lines = []
    current_line = ''
    for char in text:
        test_line = current_line + char
        if font.getlength(test_line) <= box_width - 10:  # 좌우 여백 고려
            current_line = test_line
        else:
            wrapped_lines.append(current_line)
            current_line = char
    if current_line:
        wrapped_lines.append(current_line)
    return wrapped_lines

def get_scaled_font_and_lines(text, box_width, box_height, font_path):
    max_font_size = 100
    min_font_size = 10

    for font_size in range(max_font_size, min_font_size, -2):
        try:
            font = ImageFont.truetype(font_path, font_size)
            lines = wrap_text_to_box(text, font, box_width)
            text_height = len(lines) * font_size

            if text_height <= box_height:
                return font, lines
        except Exception as e:
            logger.warning(f"폰트 크기 {font_size} 처리 중 오류: {e}")
            continue

    # 실패 → 최소 폰트 크기 적용
    try:
        font = ImageFont.truetype(font_path, min_font_size)
        lines = wrap_text_to_box(text, font, box_width)
        return font, lines
    except Exception as e:
        logger.error(f"기본 폰트 로드 실패: {e}")
        # 폰트 로드 실패 시 기본 폰트 사용
        font = ImageFont.load_default()
        lines = [text[:20] + "..."]  # 간단한 대체 텍스트
        return font, lines

def process_single_image_visualization(img, ocr_results_for_img, api_results_for_img, img_title="Document"):
    """단일 이미지에 대한 시각화 처리"""
    
    # 한글 폰트 경로 (환경별 설정)
    font_paths = [
        "/usr/share/fonts/truetype/nanum/NanumGothic.ttf",  # Colab/Ubuntu
        "/System/Library/Fonts/AppleGothic.ttf",           # macOS
        "C:/Windows/Fonts/malgun.ttf"                      # Windows
    ]
    
    font_path = None
    for path in font_paths:
        if os.path.exists(path):
            font_path = path
            break
    
    if not font_path:
        logger.warning("한글 폰트를 찾을 수 없습니다. 기본 폰트를 사용합니다.")
        font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
    
    # 캔버스 생성 (원본 이미지와 동일 크기, 흰 배경)
    canvas_height, canvas_width = img.shape[:2]
    canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255  # 흰색 배경
    
    # OCR 및 API 바운딩 박스 그리기
    all_results = ocr_results_for_img + api_results_for_img
    for result in all_results:
        x1, y1, x2, y2 = result['coordinates']
        cv2.rectangle(canvas, (x1, y1), (x2, y2), (0, 0, 0), 2)
    
    # OpenCV → PIL 변환
    canvas_pil = Image.fromarray(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(canvas_pil)
    
    # OCR 텍스트 출력
    for result in ocr_results_for_img:
        x1, y1, x2, y2 = result['coordinates']
        box_width = x2 - x1
        box_height = y2 - y1
        text = result['text'].replace('\n', ' ')
        font, lines = get_scaled_font_and_lines(text, box_width, box_height, font_path)

        for i, line in enumerate(lines):
            draw.text((x1 + 5, y1 + i * font.size), line, font=font, fill=(0, 0, 0))

    # API 텍스트 출력
    for result in api_results_for_img:
        x1, y1, x2, y2 = result['coordinates']
        box_width = x2 - x1
        box_height = y2 - y1
        description = result['description'].replace('\n', ' ')
        font, lines = get_scaled_font_and_lines(description, box_width, box_height, font_path)

        for i, line in enumerate(lines):
            draw.text((x1 + 5, y1 + i * font.size), line, font=font, fill=(0, 0, 0))
    
    # 원본 이미지와 처리된 이미지 비교 출력
    original_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    canvas_np = np.array(canvas_pil)
    
    # 이미지 높이와 너비
    height, width, _ = original_rgb.shape
    
    # 검정색 구분선 생성 (세로 5px)
    gap_width = 5
    black_line = np.zeros((height, gap_width, 3), dtype=np.uint8)
    
    # 두 이미지 가로로 이어붙이기: [원본 | 검정줄 | 처리본]
    combined_image = np.hstack((original_rgb, black_line, canvas_np))
    
    # 시각화
    plt.figure(figsize=(20, 10))
    plt.imshow(combined_image)
    plt.axis('off')
    
    # 제목
    plt.text(width // 2, 30, f'Original: {img_title}', fontsize=16, fontweight='bold',
             ha='center', va='top', color='white', bbox=dict(facecolor='black', alpha=0.7, pad=5))
    
    plt.text(width + gap_width + width // 2, 30, f'Processed: {img_title}', fontsize=16, fontweight='bold',
             ha='center', va='top', color='white', bbox=dict(facecolor='black', alpha=0.7, pad=5))
    
    plt.show()
    
    return canvas_np

# ============================================
# 🖼️ 다중 이미지 처리
# ============================================

if all_images_loaded:
    logger.info("🖼️ 다중 이미지 시각화 시작...")
    
    # 이미지별로 OCR 및 API 결과 그룹화
    for img_data in all_images_loaded:
        img_idx = img_data['index']
        original_filename = img_data['original_filename']
        page_number = img_data['page_number']
        source_type = img_data['source_type']
        img = img_data['img']
        
        # 제목 생성
        if source_type == 'pdf':
            img_title = f"{original_filename} (페이지 {page_number})"
        else:
            img_title = original_filename
        
        logger.info(f"📄 이미지 {img_idx + 1} 시각화 중: {img_title}")
        
        # 해당 이미지의 OCR 결과 필터링
        img_ocr_results = []
        for result in ocr_results:
            if 'image_index' in result and result['image_index'] == img_idx:
                img_ocr_results.append(result)
            elif 'image_index' not in result and img_idx == 0:
                # 단일 이미지 형식이면서 첫 번째 이미지인 경우
                img_ocr_results.append(result)
        
        # 해당 이미지의 API 결과 필터링
        img_api_results = []
        for result in api_results:
            if 'image_index' in result and result['image_index'] == img_idx:
                img_api_results.append(result)
            elif 'image_index' not in result and img_idx == 0:
                # 단일 이미지 형식이면서 첫 번째 이미지인 경우
                img_api_results.append(result)
        
        logger.info(f"   OCR 결과: {len(img_ocr_results)}개, API 결과: {len(img_api_results)}개")
        
        # 시각화 처리
        if len(img_ocr_results) > 0 or len(img_api_results) > 0:
            processed_canvas = process_single_image_visualization(
                img, img_ocr_results, img_api_results, img_title
            )
        else:
            logger.info(f"   처리할 OCR/API 결과가 없어 시각화를 건너뜁니다.")
    
    logger.info("✅ 다중 이미지 시각화 완료!")

# ============================================
# 🖼️ 단일 이미지 처리 (기존 호환성 유지)
# ============================================
elif img is not None:
    logger.info("🖼️ 단일 이미지 시각화 시작...")
    
    # 한글 폰트 경로 (환경별 설정)
    font_paths = [
        "/usr/share/fonts/truetype/nanum/NanumGothic.ttf",  # Colab/Ubuntu
        "/System/Library/Fonts/AppleGothic.ttf",           # macOS  
        "C:/Windows/Fonts/malgun.ttf"                      # Windows
    ]
    
    font_path = None
    for path in font_paths:
        if os.path.exists(path):
            font_path = path
            break
    
    if not font_path:
        logger.warning("한글 폰트를 찾을 수 없습니다. 기본 폰트를 사용합니다.")
        font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
    
    # 캔버스 생성
    canvas_height, canvas_width = img.shape[:2]
    canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255
    
    # 바운딩 박스 그리기
    for result in ocr_results + api_results:
        x1, y1, x2, y2 = result['coordinates']
        cv2.rectangle(canvas, (x1, y1), (x2, y2), (0, 0, 0), 2)
    
    # OpenCV → PIL 변환
    canvas_pil = Image.fromarray(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(canvas_pil)
    
    # OCR 텍스트 출력
    for result in ocr_results:
        x1, y1, x2, y2 = result['coordinates']
        box_width = x2 - x1
        box_height = y2 - y1
        text = result['text'].replace('\n', ' ')
        font, lines = get_scaled_font_and_lines(text, box_width, box_height, font_path)

        for i, line in enumerate(lines):
            draw.text((x1 + 5, y1 + i * font.size), line, font=font, fill=(0, 0, 0))

    # API 텍스트 출력
    for result in api_results:
        x1, y1, x2, y2 = result['coordinates']
        box_width = x2 - x1
        box_height = y2 - y1
        description = result['description'].replace('\n', ' ')
        font, lines = get_scaled_font_and_lines(description, box_width, box_height, font_path)

        for i, line in enumerate(lines):
            draw.text((x1 + 5, y1 + i * font.size), line, font=font, fill=(0, 0, 0))
    
    # 원본 이미지와 처리된 이미지 비교 출력
    original_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    canvas_np = np.array(canvas_pil)
    
    # 이미지 높이와 너비
    height, width, _ = original_rgb.shape
    
    # 검정색 구분선 생성 (세로 5px)
    gap_width = 5
    black_line = np.zeros((height, gap_width, 3), dtype=np.uint8)
    
    # 두 이미지 가로로 이어붙이기: [원본 | 검정줄 | 처리본]
    combined_image = np.hstack((original_rgb, black_line, canvas_np))
    
    # 최종 출력
    plt.figure(figsize=(20, 10))
    plt.imshow(combined_image)
    plt.axis('off')
    
    # 제목
    plt.text(width // 2, 30, 'Original Document Image', fontsize=16, fontweight='bold',
             ha='center', va='top', color='white', bbox=dict(facecolor='black', alpha=0.7, pad=5))
    
    plt.text(width + gap_width + width // 2, 30, '1st Processed Document Prototype', fontsize=16, fontweight='bold',
             ha='center', va='top', color='white', bbox=dict(facecolor='black', alpha=0.7, pad=5))
    
    plt.show()
    
    logger.info("✅ 단일 이미지 시각화 완료!")

else:
    logger.warning("⚠️ 시각화할 이미지가 없습니다. LAM 분석을 먼저 실행해주세요.")

## 파이선 딕셔너리 JSON 변환 컴포넌트

향후 구현 예정